In [2]:
# Atree is the matrix 
# Atree[::2,::2,::2]
# ADD S- to make search more efficient DONE
# Print Color of RGB or box of the color that specific ■ DONE
# Do not go duplicates folder DONE
# search through both DONE
(
# create excel or word documnet to show how many files and do prifiling within each case
# once you read and unravel
# Save all considered files under one of these three
# If there are multiple finals then rename them as SS-Segementations-n+1.nii.gz
# SS-Case-34-Seeds-2-Label.nii.gz  xN
# SS-Case-34-Segmentation-7-Label.nii.gz  xM
# SS-Case-34-Final-Label.nii.gz 
# if s-final date of mofied is > s-finalseg by 3 days: WARN*
# Create snapshot as final golden standard of quality assurance
# Want to potentially run this code often because we may be correct some of the segmentations
# Also may want to save these to a different location than D-Images
# MAKE SURE THAT EVERY CASE THAT IS SUPPOSED TO READ IS READ WHICH COULD BE AN ERROR IF THEY ARE OVERLAPPING
# Print number of files/folders not processed and proccessed 
# Print and and number the files that are proccessed themselves 
# Indent 
# Make font courier new
# update outputs frequently
# save outputs to two different locations
)
# COLORS: DONE
# Aseed = (0.8470588, 0.396078, 0.309803) DONE
# Vseed = (0, 0.592156, 0.807843) DONE
# Miscellaneous = (.90196, 0.86274, .27451) DONE


()

In [3]:
# QUESTIONS
# Change all .nrrd files to nii.gz
# Save all images to separate case folders like the other code
# Change all segments to Aseed and Vseed (get rid of Misc?) THEY ARE LABELS DUH 
# 
# ^^^ Step above requires deleting Misc (JUST SAVE A AND V) IGNORE MISC
# Where is code to show mayavi viewer quickly 
#
# Write to excel first 
# 
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#


In [4]:
# MOST CURRENT STATE OF CODE 11/1/2023 9:15 pm
# TESTING GOLD STANDARD NEED TO ADD VISUALIZATION AND PRINT TO WORD DOCUMENT (ADD SEARCH THROUGH ALL CASES FUNCTION TO BOTTOM OF THIS CODE)
# INTEGRATE (#GOLD STANDARD TO SAVE IMAGE OF FIGURE IN GIANT VIEWER WITH NII.GZ) to code mention above ^^^^^^^^
# COME UP WITH A WAY TO FIND THE Aseed and Vseed using some sort of indicator given the zoo diversity using these aspects(color, size, name)
# CREATE RAVELED_A = Aseed SEGMENT 
# CREATE RAVELED_V = Vseed SEGMENT 
# Show raveled Aseed as Blue
# Show raveled Vseed as Red
# Put final word document in reports
# at the top make a comment that makes the code easy to use and understandable
# make sure code works and classifys files into three categories
# pick the last final index and everything else renamed to segmentation
# using counting it should be segmentations and codes
# add red (SS-segm.nii.gz) to rename

## NEED TO UNRAVEL BEFORE I CAN ADD VISUALIZATION TO CODE OR TALK TO SOHEIL

In [5]:
# SEARCH THROUGH ALL CASES FUNCTION

# def search_and_process(base_path):
#     for root, dirs, files in os.walk(base_path):
#         for dir_name in dirs:
#             if dir_name == "1-Duplicates":
#                 continue
#             if dir_name.startswith("Case-"):
#                 for sub_root, sub_dirs, sub_files in os.walk(os.path.join(root, dir_name)):
#                     for sub_dir_name in sub_dirs:
#                         if sub_dir_name.startswith("88-"):
#                             process_nrrd_files_in_folder(os.path.join(sub_root, sub_dir_name))
# 

In [6]:
# MOST CURRENT STATE OF CODE 11/3/2023 1:03 PM
# Create Voxel Counts
# Now try to show first segmentation
# Do so by unraveleing
# that means picking the segments by some sort of indication (voxel value, color, name)
# add dimensions and spacing of the image spacing
# continue 
# Add Date of Creation or date of modification 

In [7]:
#1 Import Modules and Define Constants

# Import necessary libraries
import os
import SimpleITK as sitk
import numpy as np
from docx import Document
from docx.shared import RGBColor
from docx.shared import Pt
import time
from datetime import datetime
import nrrd
from mayavi import mlab
from docx.shared import Inches, Pt
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT



COLORS_DICT = {
    'blue': (79, 129, 189),
    'green': (155, 187, 89),
    'red': (250, 0, 0,),
    'orange': (237, 125, 49)
    # Add more colors as needed
}

# Initialize a Document object for Word file output
doc = Document()
# doc = Document("z:/Q- Figures&Papers/1-T32-Poster/Nate/CaseFileCorrection/CaseFileCorrection.docx")

In [8]:
#2 Function to write data to Word document
def write_to_word(text, bold=False, indent_level=0, font_color=None):
    paragraph = doc.add_paragraph(text)
    paragraph_format = paragraph.paragraph_format
    paragraph_format.space_after = Pt(0)
    paragraph_format.space_before = Pt(0)
    paragraph_format.left_indent = Pt(12 * indent_level)
    run = paragraph.runs[0]
    if bold:
        run.bold = True
    font = run.font
    font.name = 'Courier New'
    font.size = Pt(12)
    if font_color:
        # Check if the font_color is a named color
        if isinstance(font_color, str) and font_color in COLORS_DICT:
            r, g, b = COLORS_DICT[font_color]
        else:
            r, g, b = font_color
        font.color.rgb = RGBColor(r, g, b)
  

In [9]:
#3 add word to image
# def add_image_to_word(image_path, width=None):
#     try:
#         # Add an image to the Word document
#         if width:
#             doc.add_picture(image_path, width=Inches(width))
#         else:
#             doc.add_picture(image_path)  # Automatically uses the image's size
#     except Exception as e:
#         print(f"An error occurred while adding the image: {e}")


def add_image_to_word(image_path, width=None):
    try:
        # Create a new paragraph for the image
        image_paragraph = doc.add_paragraph()
        image_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

        # Add an image to the Word document
        if width:
            run = image_paragraph.add_run()
            run.add_picture(image_path, width=Inches(width))
        else:
            run = image_paragraph.add_run()
            run.add_picture(image_path)  # Automatically uses the image's size

    except Exception as e:
        print(f"An error occurred while adding the image: {e}")



In [10]:
#4 Function to Get Image Details
def print_image_details(image, data, full_item_path):
    # Extracting size and spacing information
    size = image.GetSize()
    spacing = image.GetSpacing()
    size_output = f"Dimensions: {size[0]} x {size[1]} x {size[2]}"
    spacing_output = f"Spacing: {spacing[0]:.2f} x {spacing[1]:.2f} x {spacing[2]:.2f} mm"
    
    # Output the size and spacing information
    print(size_output)
    print(spacing_output)
    write_to_word(size_output, indent_level=5, font_color='orange')
    write_to_word(spacing_output, indent_level=5, font_color='orange')

In [11]:
#5 Extract segment information for labels
def extract_segments_info_for_labels(file_path):
    try:
        image = sitk.ReadImage(file_path)
        data = sitk.GetArrayFromImage(image)
        unique_values = np.unique(data)
        segment_names = [f"Segment_{value}" for value in unique_values]
        label_values = list(unique_values)
        num_segments = len(label_values)
        voxel_counts = {value: np.sum(data == value) for value in unique_values}

        return num_segments, segment_names, label_values, voxel_counts
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        write_to_word(f"An error occurred: {str(e)}")
        return None, None, None

In [12]:
#6 Map Color to Label Value
def map_color_to_label(metadata, color):
    # Convert color tuple from 0-1 to 0-255 range for comparison
    color_255 = tuple(int(c * 255) for c in color)
    # Search in metadata for the label with the given color
    for key, value in metadata.items():
        if key.endswith("_Color"):
            # Extract the RGB values and convert to int in range 0-255
            meta_color = tuple(int(float(v) * 255) for v in value.split())
            if meta_color == color_255:
                # Find the label value associated with this color
                label_key = key.replace('_Color', '_LabelValue')
                return int(metadata[label_key])
    return None

In [13]:
#7 Check segment names
def check_segment_names(metadata, case_path):
    segment_names = []
    for key in sorted(metadata.keys()):
        if key.endswith("_Name"):
            segment_names.append(metadata[key])

    # if len(segment_names) > 3:
    #     print(f"Warning: More than three segments found in {case_path}. Skipping file.")
    #     return False

    if len(segment_names) >= 2:
        if not segment_names[0].startswith('A') or not segment_names[1].startswith('V'):
            print(f"Warning: The segments may be named wrong for {case_path}")
    else:
        print(f"Warning: Not enough segments found in {case_path}")

    return True

In [14]:
#8 Visualization

def visualize_nrrd_and_save_snapshot(case_path, snapshot_directory):
    # try:
        avmlabel, metadata = nrrd.read(case_path)

        OP = 1
        DecimationFactor = 2
        use_label_value = case_path.endswith('label.nrrd') or case_path.endswith('Label.nrrd')

        if use_label_value:
            # When using label values, set the segments based on the label value
            Unraveled_A = (avmlabel == 1).astype(int)
            Unraveled_V = (avmlabel == 2).astype(int)
            color_a = (0, 0.592157, 0.807843)  # Assuming these are your default colors
            color_v = (0.847059, 0.396078, 0.309804)
        else:
            if not check_segment_names(metadata, case_path):
                return
            # Define the colors for 'A' and 'V' segments
            color_a = (0, 0.592157, 0.807843)  # Assuming these are your default colors
            color_v = (0.847059, 0.396078, 0.309804)
        # Find the label values that correspond to the colors
            label_a = map_color_to_label(metadata, color_a)
            label_v = map_color_to_label(metadata, color_v)

            Unraveled_A = (avmlabel == label_a).astype(int)
            Unraveled_V = (avmlabel == label_v).astype(int)

        # Create Unraveled seagments for 'A' and 'V'
        

        # Now use 'Unraveled_A' and 'Unraveled_V' to visualize in Mayavi
        figure_name = os.path.basename(case_path).split('.')[0]
        fig = mlab.figure(figure_name, bgcolor=(1, 1, 1))

        # Visualize the Unraveled_A segment
        # DecimatedUnraveled_A = Unraveled_A[::DecimationFactor, ::DecimationFactor, ::DecimationFactor]
        # if any(DecimatedUnraveled_A)
        # mlab.contour3d(DecimatedUnraveled_A,
        #             contours=[0.5], color=color_a, opacity=OP)

        if Unraveled_A.any():
            mlab.contour3d(Unraveled_A[::DecimationFactor, ::DecimationFactor, ::DecimationFactor],
                    contours=[0.5], color=color_a, opacity=OP)

        # Visualize the Unraveled_V segment
        if Unraveled_V.any():
            mlab.contour3d(Unraveled_V[::DecimationFactor, ::DecimationFactor, ::DecimationFactor],
                    contours=[0.5], color=color_v, opacity=OP)

        # Camera and snapshot settings
        camera = mlab.gcf().scene.camera
        camera.position = [107, 140, 613]
        camera.focal_point = [175, 128, 148]
        camera.view_up = [0, 1, 0]
        camera.clipping_range = (1, 1000)

   
        path_parts = case_path.split(os.sep)  # Split the path into parts
        case_index = [i for i, part in enumerate(path_parts) if part.startswith('Case-')][0]  # Find the index of the part that starts with 'Case-'
        case_name = path_parts[case_index]  # Extract the case name

        # Constructing the directory and file names
        case_directory_name = case_name  # Use the case name as the directory name
        final_file_name = f"{case_name}-{path_parts[-1].replace('.nrrd', '.png')}"  # Construct the file name

        # Directory creation and snapshot saving
        case_directory_path = os.path.join(snapshot_directory, case_directory_name)
        if not os.path.exists(case_directory_path):
            os.makedirs(case_directory_path)

        snapshot_path = os.path.join(case_directory_path, final_file_name) 
        mlab.savefig(snapshot_path)
        
        mlab.close()
        print(f"Snapshot saved to {snapshot_path}")
        return case_path, snapshot_path 

    



        # mlab.savefig(snapshot_path)
        
        # mlab.close()
        # print(f"Snapshot saved to {snapshot_path}")
    
    # except Exception as e:
    #     print(f"An error occurred while processing {case_path}:") 
        
        # Optionally, close the figure if it's open and an error occurs
        # mlab.close()

In [15]:
#9 Print Filtered Nrrd Header

def print_filtered_nrrd_header(image, data=None):
    try:
        keys = image.GetMetaDataKeys()
        segment_count = 0
        segment_data = {}
        
        for key in keys:
            if "Segment" in key:
                segment_info = key.split('Segment')[-1]
                if segment_info[0].isdigit():
                    seg_num, attribute = segment_info.split('_', 1)
                    if seg_num not in segment_data:
                        segment_data[seg_num] = {}
                        segment_count += 1
                    if attribute in ['Color', 'ID', 'LabelValue', 'Name']:
                        value = image.GetMetaData(key)
                        segment_data[seg_num][attribute] = value
        
        output_text = f"Number of Segments: {segment_count}"
        print(output_text)
        write_to_word(output_text, indent_level=8, font_color='orange')

        if data is not None:
            total_voxel_count = f"{np.sum(data >= 0):,}"  # Sum of all voxels in the image
            output_text = f"Total Voxel Count: {total_voxel_count}"
            print(output_text)
            write_to_word(output_text, indent_level=8, font_color='orange')

        for seg_num, attributes in segment_data.items():
            segment_name_text = f"Segment {seg_num} Name: {attributes.get('Name', 'N/A')} ■"  # Concatenate the '■' character here
            segment_id = f"Segment {seg_num} ID: {attributes.get('ID', 'N/A')}"
            segment_color = f"Segment {seg_num} Color: {attributes.get('Color', 'N/A')}"
            segment_label_value = f"Segment {seg_num} LabelValue: {attributes.get('LabelValue', 'N/A')}"

            if attributes.get('Color', 'N/A') != 'N/A':
                color_value = attributes.get('Color', 'N/A')
                r, g, b = [int(float(x) * 255) for x in color_value.split()]

            # Split the segment name write into two: one for the name and one for the '■' with color.
            write_to_word(segment_name_text, indent_level=10, font_color=(r,g,b))  # Write the segment name without '■'
            
            # Add other attributes
            write_to_word(segment_id, indent_level=10)
            write_to_word(segment_color, indent_level=10)
            write_to_word(segment_label_value, indent_level=10)
            label_value = attributes.get('LabelValue')

            if label_value:
                label_value = int(label_value)  # Ensure it's an integer
                voxel_count = f"{np.sum(data == label_value):,}"  # Count and format with commas
                voxel_count_text = f"Segment {seg_num} Voxel Count: {voxel_count}"
                write_to_word(voxel_count_text, indent_level=10)

            write_to_word(" ")
    except Exception as e:
        output_text = f"An error occurred: {str(e)}"
        print(output_text)
        write_to_word(output_text)

In [16]:
#10 Process Nrrd File in folder 
def process_nrrd_files_in_folder(folder_path):

    files_and_subdirs = os.listdir(folder_path)
    output_text = f"Folder Path: {folder_path}"
    print(output_text)
    write_to_word(output_text, bold=True)
    write_to_word(" ")

    not_processed = [item for item in files_and_subdirs if not (item.startswith("S-") and item.endswith('.nrrd'))]
    if not_processed:
        output_text = f"Files/Folders NOT being processed: {len(not_processed)}"
        print(output_text)
        write_to_word(output_text, bold=True, indent_level=5, font_color='blue')
        for index, item in enumerate(not_processed, 1): # Start the index from 1
            output_text = f"{index}. {item}" # Replace dashes with numbers
            print(output_text)
            write_to_word(output_text, indent_level=8, font_color='blue')
        write_to_word(" ")
    else:
        output_text = "All files in this folder will be processed."
        print(output_text)
        write_to_word(output_text, bold=True, indent_level=10, font_color='blue')

    to_process = [item for item in files_and_subdirs if item.startswith("S-") and item.endswith('.nrrd')]
    if to_process:
        output_text = f"Files being processed: {len(to_process)}" # Number of items being processed
        print(output_text)
        write_to_word(output_text, bold=True, indent_level=5, font_color='blue')
        
        for index, item in enumerate(to_process, 1): # Start the index from 1
            output_text = f"{index}. {item}" 
            print(output_text)
            write_to_word(output_text, indent_level=8, font_color='blue')
        write_to_word(" ")

    for item in files_and_subdirs:
        if item.startswith("S-"):
            full_item_path = os.path.join(folder_path, item)
            if os.path.isfile(full_item_path) and item.endswith('.nrrd'):
                creation_time = os.path.getctime(full_item_path)
                formatted_creation_time = datetime.fromtimestamp(creation_time).strftime('%Y/%m/%d %H:%M')
                image = sitk.ReadImage(full_item_path)
                data = sitk.GetArrayFromImage(image)

                # Determine the font color
                if "label" in item.lower():
                    font_color_val = 'green'
                else:
                    font_color_val = None  # default color
                output_text = f"Processing: {item}"
                output_filename_NIFTI = item.replace("S-Final", "SS-Final").replace(".nrrd", ".nii.gz")
                output_filename_save = "→ ", output_filename_NIFTI
                print(output_text)
                write_to_word(output_text, bold=True, indent_level=5)
                write_to_word(output_filename_save, bold=True, indent_level=6, font_color='red')
                write_to_word(f"Created on: {formatted_creation_time}", indent_level=5, font_color='orange')
                image = sitk.ReadImage(full_item_path)
                data = sitk.GetArrayFromImage(image)
                print_image_details(image, data, full_item_path)
                write_to_word(' ')
                


                new_case_path, snapshot_directory_of_png = visualize_nrrd_and_save_snapshot(full_item_path, snapshot_directory)
                add_image_to_word(snapshot_directory_of_png, width=1)     



                if "label" in item.lower():
                    num_segments, segment_names, label_values, voxel_counts = extract_segments_info_for_labels(full_item_path)
                    if num_segments is not None and num_segments > 0:
                        output_text = f"Number of Segments: {num_segments}"
                        print(output_text)
                        write_to_word(output_text, indent_level=8, font_color=font_color_val)
                        for i in range(num_segments):
                            segment_output = [
                                f"Segment {i + 1}:",
                                f"  Name: {segment_names[i]}",
                                f"  Label Value: {label_values[i]}"
                                f"  Voxel Count: {format(voxel_counts[label_values[i]], ',')}"
                            ]
                            for line in segment_output:
                                print(line)
                                write_to_word(line, indent_level=10, font_color=font_color_val)
                        write_to_word(" ")

                            
                else:
                    print_filtered_nrrd_header(image, data)


In [17]:
#10.5 Search and Process Files For specific files
def search_and_process(base_path, max_cases=1):
    processed_cases = 0  # Counter to keep track of processed cases
    
    for root, dirs, files in os.walk(base_path):
        for dir_name in dirs:
            if processed_cases >= max_cases:
                break  # Exit the loop if max_cases is reached
            if dir_name == "1-Duplicates":
                continue
            if dir_name.startswith("Case-"):
                if dir_name[5:7] in ['77']:
                # if int(dir_name[5:7]) in range(10:20):
                    for sub_root, sub_dirs, sub_files in os.walk(os.path.join(root, dir_name)):
                        for sub_dir_name in sub_dirs:
                            if sub_dir_name.startswith("88-"):
                                process_nrrd_files_in_folder(os.path.join(sub_root, sub_dir_name))
                                processed_cases += 1  # Increment the counter
                                if processed_cases >= max_cases:
                                    return 


In [18]:
#11 Search and Process Files 
def search_and_process(base_path, max_cases=1):
    processed_cases = 0  # Counter to keep track of processed cases
    
    for root, dirs, files in os.walk(base_path):
        for dir_name in dirs:
            if processed_cases >= max_cases:
                break  # Exit the loop if max_cases is reached
            if dir_name == "1-Duplicates":
                continue
            if dir_name.startswith("Case-"):
                for sub_root, sub_dirs, sub_files in os.walk(os.path.join(root, dir_name)):
                    for sub_dir_name in sub_dirs:
                        if sub_dir_name.startswith("88-"):
                            process_nrrd_files_in_folder(os.path.join(sub_root, sub_dir_name))
                            processed_cases += 1  # Increment the counter
                            if processed_cases >= max_cases:
                                return 


In [3]:
#100 GOLD STANDARD CODE ALL IN ONE

# Import necessary libraries
import os
import SimpleITK as sitk
import numpy as np
from docx import Document
import time
from datetime import datetime
import nrrd
from mayavi import mlab
from docx.shared import Inches, Pt, RGBColor
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
import re
from docx.oxml.ns import qn
from docx.enum.text import WD_COLOR_INDEX
from docx.oxml import OxmlElement

HIGHLIGHT_COLORS = {
    'yellow': WD_COLOR_INDEX.YELLOW,
    'green': WD_COLOR_INDEX.BRIGHT_GREEN,
    'cyan': WD_COLOR_INDEX.TURQUOISE,
    'magenta': WD_COLOR_INDEX.PINK,
    'blue': WD_COLOR_INDEX.BLUE,
    'red': WD_COLOR_INDEX.RED,
    'none': WD_COLOR_INDEX.AUTO  # No highlighting
}


def add_horizontal_line(doc):
    """Adds a horizontal line to the Word document."""
    # Create a new paragraph for the horizontal line
    para = doc.add_paragraph()
    para_format = para.paragraph_format
    para_format.space_after = Pt(0)  # Reduce space after the paragraph
    para_format.space_before = Pt(0)
    p = para._p  # Access the underlying XML element of the paragraph
    pPr = p.get_or_add_pPr()  # Access paragraph properties

    # Create a border element
    pBdr = OxmlElement('w:pBdr')

    # Create a bottom border element
    bottomBdr = OxmlElement('w:bottom')
    bottomBdr.set(qn('w:val'), 'single')
    bottomBdr.set(qn('w:sz'), '4')  # Size of the border
    bottomBdr.set(qn('w:space'), '1')  # Space between border and text
    bottomBdr.set(qn('w:color'), 'auto')  # Border color

    # Add the bottom border to the border element
    pBdr.append(bottomBdr)

    # Add the border element to the paragraph properties
    pPr.append(pBdr)
def set_section_columns(section, num_columns, space_cm, separator=True):
    """ Set up the number of columns in a Word document section.
    
    Args:
    section (docx.section.Section): The section to modify.
    num_columns (int): The number of columns to set.
    space_cm (float): The spacing between columns in centimeters.
    separator (bool): Whether to add a line between columns.
    """
    sectPr = section._sectPr
    cols = sectPr.xpath('./w:cols')[0]
    cols.set(qn('w:num'), str(num_columns))
    cols.set(qn('w:space'), str(int(space_cm * 20)))  # Word space units are 1/20th of a point
    if separator:
        cols.set(qn('w:sep'), '1')
    else:
        cols.set(qn('w:sep'), '0')

COLORS_DICT = {
    'blue': (79, 129, 189),
    'green': (155, 187, 89),
    'red': (250, 0, 0,),
    'orange': (237, 125, 49),
    'white': (255, 255, 255)
    # Add more colors as needed
}

# Initialize a Document object for Word file output
doc = Document()
# doc = Document("z:/Q- Figures&Papers/1-T32-Poster/Nate/CaseFileCorrection/CaseFileCorrection.docx")
num_columns = 3  # or 3 for three columns
space_between_columns_cm = .5  # Adjust the spacing as necessary
set_section_columns(doc.sections[0], num_columns, space_between_columns_cm)


#2 Function to write data to Word document
##############################################################################################################
def write_to_word(text, bold=False, indent_level=0, font_color=None, alignment=None, highlight=None, label_file=False):
    paragraph = doc.add_paragraph()
    paragraph_format = paragraph.paragraph_format
    paragraph_format.space_after = Pt(0)
    paragraph_format.space_before = Pt(0)
    paragraph_format.left_indent = Pt(12 * indent_level)
    if label_file and 'Voxel Count' in text:
        font_color = 'green'

    # Check if the text contains a colon and a highlight color is specified
    if ':' in text and highlight:
        # Split the text into label and values
        label, values = text.split(':', 1)
        label_run = paragraph.add_run(label + ':')
        label_run.font.name = 'Courier New'
        label_run.font.size = Pt(6)
        if font_color:
            # Apply the font color if specified
            r, g, b = COLORS_DICT.get(font_color, (0, 0, 0))
            label_run.font.color.rgb = RGBColor(r, g, b)
        
        # Now create a new run for the values which will be highlighted
        values_run = paragraph.add_run(values)
        values_run.font.name = 'Courier New'
        values_run.font.size = Pt(6)
        if bold:
            values_run.bold = True

        # Set the highlight color for values
        if highlight in HIGHLIGHT_COLORS:
            values_run.font.highlight_color = HIGHLIGHT_COLORS[highlight]
            if HIGHLIGHT_COLORS[highlight] == WD_COLOR_INDEX.BLUE:
                # If highlight is blue, set font color to white for values
                values_run.font.color.rgb = RGBColor(255, 255, 255)
    else:
        # If there's no colon or no highlight, just treat it as a normal run
        run = paragraph.add_run(text)
        run.font.name = 'Courier New'
        run.font.size = Pt(6)
        if bold:
            run.bold = True

        # Set the font color
        if font_color:
            if isinstance(font_color, tuple) and len(font_color) == 3:
                run.font.color.rgb = RGBColor(*font_color)
            else:
                r, g, b = COLORS_DICT.get(font_color, (0, 0, 0))
                run.font.color.rgb = RGBColor(r, g, b)

        # Set the highlight color for the run
        if highlight in HIGHLIGHT_COLORS:
            run.font.highlight_color = HIGHLIGHT_COLORS[highlight]
            if HIGHLIGHT_COLORS[highlight] == WD_COLOR_INDEX.BLUE:
                # If highlight is blue, set font color to white
                run.font.color.rgb = RGBColor(255, 255, 255)

    # Set paragraph alignment if specified
    if alignment:
        paragraph.alignment = alignment

#3
def add_image_to_word(image_path, width=None):
    try:
        # Create a new paragraph for the image
        image_paragraph = doc.add_paragraph()
        image_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

        # Add an image to the Word document
        if width:
            run = image_paragraph.add_run()
            run.add_picture(image_path, width=Inches(width))
        else:
            run = image_paragraph.add_run()
            run.add_picture(image_path)  # Automatically uses the image's size

    except Exception as e:
        print(f"An error occurred while adding the image: {e}")

#4 Function to Get Image Details
def print_image_details(image, data, full_item_path):
    # Extracting size and spacing information
    size = image.GetSize()
    spacing = image.GetSpacing()
    size_output = f"Dimensions: {size[0]} x {size[1]} x {size[2]}"
    spacing_output = f"Spacing: {spacing[0]:.2f} x {spacing[1]:.2f} x {spacing[2]:.2f} mm"
    
    # Output the size and spacing information
    print(size_output)
    print(spacing_output)
    write_to_word(size_output, indent_level=0, alignment=WD_PARAGRAPH_ALIGNMENT.CENTER, highlight='yellow')
    write_to_word(spacing_output, indent_level=0, alignment=WD_PARAGRAPH_ALIGNMENT.CENTER, highlight='yellow')

#5 Extract segment information for labels
def extract_segments_info_for_labels(file_path):
    try:
        image = sitk.ReadImage(file_path)
        data = sitk.GetArrayFromImage(image)
        unique_values = np.unique(data)
        segment_names = [f"Segment_{value}" for value in unique_values]
        label_values = list(unique_values)
        num_segments = len(label_values)
        voxel_counts = {value: np.sum(data == value) for value in unique_values}

        return num_segments, segment_names, label_values, voxel_counts
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        write_to_word(f"An error occurred: {str(e)}")
        return None, None, None
    
#6 Map Color to Label Value
def map_color_to_label(metadata, color):
    # Convert color tuple from 0-1 to 0-255 range for comparison
    color_255 = tuple(int(c * 255) for c in color)
    # Search in metadata for the label with the given color
    for key, value in metadata.items():
        if key.endswith("_Color"):
            # Extract the RGB values and convert to int in range 0-255
            meta_color = tuple(int(float(v) * 255) for v in value.split())
            if meta_color == color_255:
                # Find the label value associated with this color
                label_key = key.replace('_Color', '_LabelValue')
                return int(metadata[label_key])
    return None

#7 Check segment names
def check_segment_names(metadata, case_path):
    segment_names = []
    for key in sorted(metadata.keys()):
        if key.endswith("_Name"):
            segment_names.append(metadata[key])

    # if len(segment_names) > 3:
    #     print(f"Warning: More than three segments found in {case_path}. Skipping file.")
    #     return False

    if len(segment_names) >= 2:
        if not segment_names[0].startswith('A') or not segment_names[1].startswith('V'):
            print(f"Warning: The segments may be named wrong for {case_path}")
    else:
        print(f"Warning: Not enough segments found in {case_path}")

    return True

#8 Visualization
def visualize_nrrd_and_save_snapshot(case_path, snapshot_directory):
    # try:
        avmlabel, metadata = nrrd.read(case_path)

        OP = 1
        DecimationFactor = 2
        use_label_value = case_path.endswith('label.nrrd') or case_path.endswith('Label.nrrd')

        if use_label_value:
            # When using label values, set the segments based on the label value
            Unraveled_A = (avmlabel == 1).astype(int)
            Unraveled_V = (avmlabel == 2).astype(int)
            color_a = (0, 0.592157, 0.807843)  # Assuming these are your default colors
            color_v = (0.847059, 0.396078, 0.309804)
        else:
            if not check_segment_names(metadata, case_path):
                return
            # Define the colors for 'A' and 'V' segments
            color_a = (0, 0.592157, 0.807843)  # Assuming these are your default colors
            color_v = (0.847059, 0.396078, 0.309804)
        # Find the label values that correspond to the colors
            label_a = map_color_to_label(metadata, color_a)
            label_v = map_color_to_label(metadata, color_v)

            Unraveled_A = (avmlabel == label_a).astype(int)
            Unraveled_V = (avmlabel == label_v).astype(int)

        # Create Unraveled seagments for 'A' and 'V'
        
        # Now use 'Unraveled_A' and 'Unraveled_V' to visualize in Mayavi
        figure_name = os.path.basename(case_path).split('.')[0]
        fig = mlab.figure(figure_name, bgcolor=(1, 1, 1))

        if Unraveled_A.any():
            mlab.contour3d(Unraveled_A[::DecimationFactor, ::DecimationFactor, ::DecimationFactor],
                    contours=[0.5], color=color_a, opacity=OP)

        # Visualize the Unraveled_V segment
        if Unraveled_V.any():
            mlab.contour3d(Unraveled_V[::DecimationFactor, ::DecimationFactor, ::DecimationFactor],
                    contours=[0.5], color=color_v, opacity=OP)

        # Camera and snapshot settings
        camera = mlab.gcf().scene.camera
        camera.position = [107, 140, 613]
        camera.focal_point = [175, 128, 148]
        camera.view_up = [0, 1, 0]
        camera.clipping_range = (1, 1000)

        path_parts = case_path.split(os.sep)  # Split the path into parts
        case_index = [i for i, part in enumerate(path_parts) if part.startswith('Case-')][0]  # Find the index of the part that starts with 'Case-'
        case_name = path_parts[case_index]  # Extract the case name

        # Constructing the directory and file names
        case_directory_name = case_name  # Use the case name as the directory name
        final_file_name = f"{case_name}-{path_parts[-1].replace('.nrrd', '.png')}"  # Construct the file name

        # Directory creation and snapshot saving
        case_directory_path = os.path.join(snapshot_directory, case_directory_name)
        if not os.path.exists(case_directory_path):
            os.makedirs(case_directory_path)

        snapshot_path = os.path.join(case_directory_path, final_file_name) 
        mlab.savefig(snapshot_path)
        
        mlab.close()
        print(f"Snapshot saved to {snapshot_path}")
        return case_path, snapshot_path 

#9 Print Filtered Nrrd Header
def print_filtered_nrrd_header(image, data=None):
    try:
        keys = image.GetMetaDataKeys()
        segment_count = 0
        segment_data = {}
        
        for key in keys:
            if "Segment" in key:
                segment_info = key.split('Segment')[-1]
                if segment_info[0].isdigit():
                    seg_num, attribute = segment_info.split('_', 1)
                    if seg_num not in segment_data:
                        segment_data[seg_num] = {}
                        segment_count += 1
                    if attribute in ['Color', 'ID', 'LabelValue', 'Name']:
                        value = image.GetMetaData(key)
                        segment_data[seg_num][attribute] = value
        
        output_text = f"Number of Segments: {segment_count}"
        print(output_text)
        write_to_word(output_text, indent_level=0, alignment=WD_PARAGRAPH_ALIGNMENT.CENTER)

        if data is not None:
            total_voxel_count = f"{np.sum(data >= 0):,}"  # Sum of all voxels in the image
            output_text = f"Total Voxel Count: {total_voxel_count}"
            print(output_text)
            # write_to_word(output_text, indent_level=8, font_color='orange')

        for seg_num, attributes in segment_data.items():
            segment_name_text = f"Segment {seg_num} Name: {attributes.get('Name', 'N/A')} ■"  # Concatenate the '■' character here
            # segment_id = f"Segment {seg_num} ID: {attributes.get('ID', 'N/A')}"
            # segment_color = f"Segment {seg_num} Color: {attributes.get('Color', 'N/A')}"
            segment_label_value = f"LabelValue: {attributes.get('LabelValue', 'N/A')}"

            if attributes.get('Color', 'N/A') != 'N/A':
                color_value = attributes.get('Color', 'N/A')
                r, g, b = [int(float(x) * 255) for x in color_value.split()]

            # Split the segment name write into two: one for the name and one for the '■' with color.
            write_to_word(segment_name_text, indent_level=0, font_color=(r,g,b), alignment=WD_PARAGRAPH_ALIGNMENT.CENTER)  # Write the segment name without '■'
            
            # Add other attributes
            # write_to_word(segment_id, indent_level=10)
            # write_to_word(segment_color, indent_level=0)
            write_to_word(segment_label_value, indent_level=0, alignment=WD_PARAGRAPH_ALIGNMENT.CENTER, highlight='yellow')
            label_value = attributes.get('LabelValue')

            if label_value:
                label_value = int(label_value)  # Ensure it's an integer
                voxel_count = f"{np.sum(data == label_value):,}"  # Count and format with commas
                voxel_count_text = f"Voxel Count: {voxel_count}"
                write_to_word(voxel_count_text, indent_level=0, alignment=WD_PARAGRAPH_ALIGNMENT.CENTER, highlight='yellow')
            write_to_word(" ")

    except Exception as e:
        output_text = f"An error occurred: {str(e)}"
        print(output_text)
        write_to_word(output_text)

#10 Process Nrrd File in folder 
def process_nrrd_files_in_folder(folder_path): 
    add_horizontal_line(doc)
    # Extract case ID from the folder path
    

# Initialize the dictionary to keep track of added images
    pattern = r"Case-\d+"
    match = re.search(pattern, folder_path)
    case_id = match.group()
    files_and_subdirs = os.listdir(folder_path)
    output_text = f"File: {case_id}"
    print(output_text)
    write_to_word(output_text, bold=True, alignment=WD_PARAGRAPH_ALIGNMENT.CENTER, highlight='red')
    # write_to_word(" ")

    not_processed = [item for item in files_and_subdirs if not (item.startswith("S-") and item.endswith('.nrrd'))]
    if not_processed:
        output_text = f"Files NOT being processed: {len(not_processed)}"
        print(output_text)
        write_to_word(output_text, bold=True, indent_level=0, font_color='blue', alignment=WD_PARAGRAPH_ALIGNMENT.CENTER)
        for index, item in enumerate(not_processed, 1): # Start the index from 1
            output_text = f"{index}. {item}" # Replace dashes with numbers
            print(output_text)
            write_to_word(output_text, indent_level=0, font_color='blue', alignment=WD_PARAGRAPH_ALIGNMENT.CENTER)
        write_to_word(" ")
    else:
        output_text = "All files in this folder will be processed."
        print(output_text)
        write_to_word(output_text, bold=True, indent_level=0, font_color='blue', alignment=WD_PARAGRAPH_ALIGNMENT.CENTER)

    to_process = [item for item in files_and_subdirs if item.startswith("S-") and item.endswith('.nrrd')]
    if to_process:
        output_text = f"Files being processed: {len(to_process)}" # Number of items being processed

        print(output_text)
        write_to_word(output_text, bold=True, indent_level=0, font_color='blue', alignment=WD_PARAGRAPH_ALIGNMENT.CENTER)
        
        for index, item in enumerate(to_process, 1): # Start the index from 1
            output_text = f"{index}. {item}" 
            print(output_text)
            # write_to_word(output_text, indent_level=0, font_color='blue')

    for item in files_and_subdirs:
        if item.startswith("S-"):
            full_item_path = os.path.join(folder_path, item)
            if os.path.isfile(full_item_path) and item.endswith('.nrrd'):
                creation_time = os.path.getctime(full_item_path)
                formatted_creation_time = datetime.fromtimestamp(creation_time).strftime('%Y/%m/%d %H:%M')
                image = sitk.ReadImage(full_item_path)
                data = sitk.GetArrayFromImage(image)

                # Determine the font color
                if "label" in item.lower():
                    font_color_val = 'green'
                else:
                    font_color_val = None  # default color
                output_text = f"{item}"
                output_filename_NIFTI = item.replace("S-", "SS-").replace(".nrrd", ".nii.gz").replace(".seg.",".")
                output_filename_save = "→ ", output_filename_NIFTI
                print(output_text)
                write_to_word(output_text, bold=True, indent_level=0, font_color='white', alignment=WD_PARAGRAPH_ALIGNMENT.CENTER, highlight='blue')
                write_to_word(output_filename_save, bold=True, indent_level=0, font_color='red', alignment=WD_PARAGRAPH_ALIGNMENT.CENTER)
                write_to_word(f"Created on: {formatted_creation_time}", indent_level=0, alignment=WD_PARAGRAPH_ALIGNMENT.CENTER)
                image = sitk.ReadImage(full_item_path)
                data = sitk.GetArrayFromImage(image)
                print_image_details(image, data, full_item_path)
                                         






                new_case_path, snapshot_directory_of_png = visualize_nrrd_and_save_snapshot(full_item_path, snapshot_directory)
                add_image_to_word(snapshot_directory_of_png, width=1)     
                
                if "label" in item.lower():
                    num_segments, segment_names, label_values, voxel_counts = extract_segments_info_for_labels(full_item_path)
                    if num_segments is not None and num_segments > 0:
                        output_text = f"Number of Segments: {num_segments}"
                        print(output_text)
                        write_to_word(output_text, indent_level=0, font_color=font_color_val, alignment=WD_PARAGRAPH_ALIGNMENT.CENTER)
                       
                        for i in range(num_segments):
                            
                            segment_output = [
                                f"Segment {i + 1}:",
                                f"Label Value: {label_values[i]} \n"
                                f"Voxel Count: {format(voxel_counts[label_values[i]], ',')}"
                            ]
                            for line in segment_output:
                                print(line)
                                write_to_word(line, indent_level=0, font_color=font_color_val, alignment=WD_PARAGRAPH_ALIGNMENT.CENTER, highlight='yellow', label_file=True)
                        
                        # write_to_word(" ")
                else:
                    print_filtered_nrrd_header(image, data)
        add_horizontal_line(doc)
        
#11 Search and Process Files 
def search_and_process(base_path, max_cases=77):
    processed_cases = 0  # Counter to keep track of processed cases
    for root, dirs, files in os.walk(base_path):
        for dir_name in dirs:
            if processed_cases >= max_cases:
                break  # Exit the loop if max_cases is reached
            if dir_name == "1-Duplicates":
                continue
            if dir_name.startswith("Case-"):
                for sub_root, sub_dirs, sub_files in os.walk(os.path.join(root, dir_name)):
                    for sub_dir_name in sub_dirs:
                        if sub_dir_name.startswith("88-"):
                            process_nrrd_files_in_folder(os.path.join(sub_root, sub_dir_name))
                            doc.save('z:/Q- Figures&Papers/1-T32-Poster/Nate/CaseFileCorrection/CaseFileCorrection(V16).docx')
                            processed_cases += 1  # Increment the counter
                            if processed_cases >= max_cases:
                                return 

 
#12 Define the directory path and call all functions previously
base_folder_path = "z:/D-Images/"
snapshot_directory = "z:/Q- Figures&Papers/1-T32-Poster/Nate/CaseFileCorrection/"
vx3 = "z:/Q- Figures&Papers/1-T32-Poster/Nate/ImageCreationFile/QC-Images/"
image_files = sorted(os.listdir(vx3))
image_paths = [os.path.join(vx3, file) for file in image_files if file.endswith('.png')]
search_and_process(base_folder_path)

# Save the document
doc.save('z:/Q- Figures&Papers/1-T32-Poster/Nate/CaseFileCorrection/CaseFileCorrection(V16).docx')

File: Case-01
Files NOT being processed: 9
1. 99-Duncan
2. 99-Soheil
3. F-1-QC.mrk.json
4. F-2-ComplicatedPoints.mrk.json
5. F-3.mrk.json
6. F-4-Soheils.mrk.json
7. Trash
8. Vx3-2.nrrd
9. Vx3.nrrd
Files being processed: 10
1. S-Final-1.nrrd
2. S-Final-2.nrrd
3. S-Final-3-label.nrrd
4. S-Final-3.nrrd
5. S-Seeds-1.nrrd
6. S-Segmentation-1.nrrd
7. S-Segmentation-2.nrrd
8. S-Segmentation-3.nrrd
9. S-Segmentation-4.nrrd
10. S-Segmentation-5.nrrd
S-Final-1.nrrd
Dimensions: 625 x 451 x 632
Spacing: 0.38 x 0.38 x 0.38 mm
Snapshot saved to z:/Q- Figures&Papers/1-T32-Poster/Nate/CaseFileCorrection/Case-01-MESAL-5010098-E3\Case-01-MESAL-5010098-E3-S-Final-1.png
Number of Segments: 3
Total Voxel Count: 178,145,000
S-Final-2.nrrd
Dimensions: 625 x 451 x 632
Spacing: 0.38 x 0.38 x 0.38 mm
Snapshot saved to z:/Q- Figures&Papers/1-T32-Poster/Nate/CaseFileCorrection/Case-01-MESAL-5010098-E3\Case-01-MESAL-5010098-E3-S-Final-2.png
Number of Segments: 3
Total Voxel Count: 178,145,000
S-Final-3-label.nrrd


TraitError: The 'scalar_data' trait of an ArraySource instance must be a 2D or 3D numpy array, but a value of array([[[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],

        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],

        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],

        ...,

        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]

In [25]:
################################################ TEMPORARY WORKING!!!!!!!!!
#100 GOLD STANDARD CODE ALL IN ONE

# Import necessary libraries
import os
import SimpleITK as sitk
import numpy as np
from docx import Document
import time
from datetime import datetime
import nrrd
from mayavi import mlab
from docx.shared import Inches, Pt, RGBColor
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
import re
from docx.oxml.ns import qn
from docx.enum.text import WD_COLOR_INDEX
from docx.oxml import OxmlElement

HIGHLIGHT_COLORS = {
    'yellow': WD_COLOR_INDEX.YELLOW,
    'green': WD_COLOR_INDEX.BRIGHT_GREEN,
    'cyan': WD_COLOR_INDEX.TURQUOISE,
    'magenta': WD_COLOR_INDEX.PINK,
    'blue': WD_COLOR_INDEX.BLUE,
    'red': WD_COLOR_INDEX.RED,
    'none': WD_COLOR_INDEX.AUTO  # No highlighting
}
def extract_case_identifier(file_name):
    pattern = r"Case-\d+"
    match = re.search(pattern, file_name)
    if match:
        return match.group()
    return None

def map_case_identifiers_to_images(image_directory):
    image_files = os.listdir(image_directory)
    case_to_image_map = {}
    for image_file in image_files:
        case_id = extract_case_identifier(image_file)
        if case_id:
            case_to_image_map[case_id] = os.path.join(image_directory, image_file)
    return case_to_image_map

case_image_map = map_case_identifiers_to_images(vx3)
def add_horizontal_line(doc):
    """Adds a horizontal line to the Word document."""
    # Create a new paragraph for the horizontal line
    para = doc.add_paragraph()
    para_format = para.paragraph_format
    para_format.space_after = Pt(0)  # Reduce space after the paragraph
    para_format.space_before = Pt(0)
    p = para._p  # Access the underlying XML element of the paragraph
    pPr = p.get_or_add_pPr()  # Access paragraph properties

    # Create a border element
    pBdr = OxmlElement('w:pBdr')

    # Create a bottom border element
    bottomBdr = OxmlElement('w:bottom')
    bottomBdr.set(qn('w:val'), 'single')
    bottomBdr.set(qn('w:sz'), '4')  # Size of the border
    bottomBdr.set(qn('w:space'), '1')  # Space between border and text
    bottomBdr.set(qn('w:color'), 'auto')  # Border color

    # Add the bottom border to the border element
    pBdr.append(bottomBdr)

    # Add the border element to the paragraph properties
    pPr.append(pBdr)
def set_section_columns(section, num_columns, space_cm, separator=True):
    """ Set up the number of columns in a Word document section.
    
    Args:
    section (docx.section.Section): The section to modify.
    num_columns (int): The number of columns to set.
    space_cm (float): The spacing between columns in centimeters.
    separator (bool): Whether to add a line between columns.
    """
    sectPr = section._sectPr
    cols = sectPr.xpath('./w:cols')[0]
    cols.set(qn('w:num'), str(num_columns))
    cols.set(qn('w:space'), str(int(space_cm * 20)))  # Word space units are 1/20th of a point
    if separator:
        cols.set(qn('w:sep'), '1')
    else:
        cols.set(qn('w:sep'), '0')

COLORS_DICT = {
    'blue': (79, 129, 189),
    'green': (155, 187, 89),
    'red': (250, 0, 0,),
    'orange': (237, 125, 49),
    'white': (255, 255, 255)
    # Add more colors as needed
}

# Initialize a Document object for Word file output
doc = Document()
# doc = Document("z:/Q- Figures&Papers/1-T32-Poster/Nate/CaseFileCorrection/CaseFileCorrection.docx")
num_columns = 3  # or 3 for three columns
space_between_columns_cm = .5  # Adjust the spacing as necessary
set_section_columns(doc.sections[0], num_columns, space_between_columns_cm)


#2 Function to write data to Word document
##############################################################################################################
def write_to_word(text, bold=False, indent_level=0, font_color=None, alignment=None, highlight=None, label_file=False):
    paragraph = doc.add_paragraph()
    paragraph_format = paragraph.paragraph_format
    paragraph_format.space_after = Pt(0)
    paragraph_format.space_before = Pt(0)
    paragraph_format.left_indent = Pt(12 * indent_level)
    if label_file and 'Voxel Count' in text:
        font_color = 'green'

    # Check if the text contains a colon and a highlight color is specified
    if ':' in text and highlight:
        # Split the text into label and values
        label, values = text.split(':', 1)
        label_run = paragraph.add_run(label + ':')
        label_run.font.name = 'Courier New'
        label_run.font.size = Pt(6)
        if font_color:
            # Apply the font color if specified
            r, g, b = COLORS_DICT.get(font_color, (0, 0, 0))
            label_run.font.color.rgb = RGBColor(r, g, b)
        
        # Now create a new run for the values which will be highlighted
        values_run = paragraph.add_run(values)
        values_run.font.name = 'Courier New'
        values_run.font.size = Pt(6)
        if bold:
            values_run.bold = True

        # Set the highlight color for values
        if highlight in HIGHLIGHT_COLORS:
            values_run.font.highlight_color = HIGHLIGHT_COLORS[highlight]
            if HIGHLIGHT_COLORS[highlight] == WD_COLOR_INDEX.BLUE:
                # If highlight is blue, set font color to white for values
                values_run.font.color.rgb = RGBColor(255, 255, 255)
    else:
        # If there's no colon or no highlight, just treat it as a normal run
        run = paragraph.add_run(text)
        run.font.name = 'Courier New'
        run.font.size = Pt(6)
        if bold:
            run.bold = True

        # Set the font color
        if font_color:
            if isinstance(font_color, tuple) and len(font_color) == 3:
                run.font.color.rgb = RGBColor(*font_color)
            else:
                r, g, b = COLORS_DICT.get(font_color, (0, 0, 0))
                run.font.color.rgb = RGBColor(r, g, b)

        # Set the highlight color for the run
        if highlight in HIGHLIGHT_COLORS:
            run.font.highlight_color = HIGHLIGHT_COLORS[highlight]
            if HIGHLIGHT_COLORS[highlight] == WD_COLOR_INDEX.BLUE:
                # If highlight is blue, set font color to white
                run.font.color.rgb = RGBColor(255, 255, 255)

    # Set paragraph alignment if specified
    if alignment:
        paragraph.alignment = alignment

#3
def add_image_to_word(image_path, width=None):
    try:
        # Create a new paragraph for the image
        image_paragraph = doc.add_paragraph()
        image_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

        # Add an image to the Word document
        if width:
            run = image_paragraph.add_run()
            run.add_picture(image_path, width=Inches(width))
        else:
            run = image_paragraph.add_run()
            run.add_picture(image_path)  # Automatically uses the image's size

    except Exception as e:
        print(f"An error occurred while adding the image: {e}")

#4 Function to Get Image Details
def print_image_details(image, data, full_item_path):
    # Extracting size and spacing information
    size = image.GetSize()
    spacing = image.GetSpacing()
    size_output = f"Dimensions: {size[0]} x {size[1]} x {size[2]}"
    spacing_output = f"Spacing: {spacing[0]:.2f} x {spacing[1]:.2f} x {spacing[2]:.2f} mm"
    
    # Output the size and spacing information
    print(size_output)
    print(spacing_output)
    write_to_word(size_output, indent_level=0, alignment=WD_PARAGRAPH_ALIGNMENT.CENTER, highlight='yellow')
    write_to_word(spacing_output, indent_level=0, alignment=WD_PARAGRAPH_ALIGNMENT.CENTER, highlight='yellow')

#5 Extract segment information for labels
def extract_segments_info_for_labels(file_path):
    try:
        image = sitk.ReadImage(file_path)
        data = sitk.GetArrayFromImage(image)
        unique_values = np.unique(data)
        segment_names = [f"Segment_{value}" for value in unique_values]
        label_values = list(unique_values)
        num_segments = len(label_values)
        voxel_counts = {value: np.sum(data == value) for value in unique_values}

        return num_segments, segment_names, label_values, voxel_counts
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        write_to_word(f"An error occurred: {str(e)}")
        return None, None, None
    
#6 Map Color to Label Value
def map_color_to_label(metadata, color):
    # Convert color tuple from 0-1 to 0-255 range for comparison
    color_255 = tuple(int(c * 255) for c in color)
    # Search in metadata for the label with the given color
    for key, value in metadata.items():
        if key.endswith("_Color"):
            # Extract the RGB values and convert to int in range 0-255
            meta_color = tuple(int(float(v) * 255) for v in value.split())
            if meta_color == color_255:
                # Find the label value associated with this color
                label_key = key.replace('_Color', '_LabelValue')
                return int(metadata[label_key])
    return None

#7 Check segment names
def check_segment_names(metadata, case_path):
    segment_names = []
    for key in sorted(metadata.keys()):
        if key.endswith("_Name"):
            segment_names.append(metadata[key])

    # if len(segment_names) > 3:
    #     print(f"Warning: More than three segments found in {case_path}. Skipping file.")
    #     return False

    if len(segment_names) >= 2:
        if not segment_names[0].startswith('A') or not segment_names[1].startswith('V'):
            print(f"Warning: The segments may be named wrong for {case_path}")
    else:
        print(f"Warning: Not enough segments found in {case_path}")

    return True

#8 Visualization
def visualize_nrrd_and_save_snapshot(case_path, snapshot_directory):
    # try:
        avmlabel, metadata = nrrd.read(case_path)

        OP = 1
        DecimationFactor = 2
        use_label_value = case_path.endswith('label.nrrd') or case_path.endswith('Label.nrrd')

        if use_label_value:
            # When using label values, set the segments based on the label value
            Unraveled_A = (avmlabel == 1).astype(int)
            Unraveled_V = (avmlabel == 2).astype(int)
            color_a = (0, 0.592157, 0.807843)  # Assuming these are your default colors
            color_v = (0.847059, 0.396078, 0.309804)
        else:
            if not check_segment_names(metadata, case_path):
                return
            # Define the colors for 'A' and 'V' segments
            color_a = (0, 0.592157, 0.807843)  # Assuming these are your default colors
            color_v = (0.847059, 0.396078, 0.309804)
        # Find the label values that correspond to the colors
            label_a = map_color_to_label(metadata, color_a)
            label_v = map_color_to_label(metadata, color_v)

            Unraveled_A = (avmlabel == label_a).astype(int)
            Unraveled_V = (avmlabel == label_v).astype(int)

        # Create Unraveled seagments for 'A' and 'V'
        
        # Now use 'Unraveled_A' and 'Unraveled_V' to visualize in Mayavi
        figure_name = os.path.basename(case_path).split('.')[0]
        fig = mlab.figure(figure_name, bgcolor=(1, 1, 1))

        if Unraveled_A.any():
            mlab.contour3d(Unraveled_A[::DecimationFactor, ::DecimationFactor, ::DecimationFactor],
                    contours=[0.5], color=color_a, opacity=OP)

        # Visualize the Unraveled_V segment
        if Unraveled_V.any():
            mlab.contour3d(Unraveled_V[::DecimationFactor, ::DecimationFactor, ::DecimationFactor],
                    contours=[0.5], color=color_v, opacity=OP)

        # Camera and snapshot settings
        camera = mlab.gcf().scene.camera
        camera.position = [107, 140, 613]
        camera.focal_point = [175, 128, 148]
        camera.view_up = [0, 1, 0]
        camera.clipping_range = (1, 1000)

        path_parts = case_path.split(os.sep)  # Split the path into parts
        case_index = [i for i, part in enumerate(path_parts) if part.startswith('Case-')][0]  # Find the index of the part that starts with 'Case-'
        case_name = path_parts[case_index]  # Extract the case name

        # Constructing the directory and file names
        case_directory_name = case_name  # Use the case name as the directory name
        final_file_name = f"{case_name}-{path_parts[-1].replace('.nrrd', '.png')}"  # Construct the file name

        # Directory creation and snapshot saving
        case_directory_path = os.path.join(snapshot_directory, case_directory_name)
        if not os.path.exists(case_directory_path):
            os.makedirs(case_directory_path)

        snapshot_path = os.path.join(case_directory_path, final_file_name) 
        mlab.savefig(snapshot_path)
        
        mlab.close()
        print(f"Snapshot saved to {snapshot_path}")
        return case_path, snapshot_path 

#9 Print Filtered Nrrd Header
def print_filtered_nrrd_header(image, data=None):
    try:
        keys = image.GetMetaDataKeys()
        segment_count = 0
        segment_data = {}
        
        for key in keys:
            if "Segment" in key:
                segment_info = key.split('Segment')[-1]
                if segment_info[0].isdigit():
                    seg_num, attribute = segment_info.split('_', 1)
                    if seg_num not in segment_data:
                        segment_data[seg_num] = {}
                        segment_count += 1
                    if attribute in ['Color', 'ID', 'LabelValue', 'Name']:
                        value = image.GetMetaData(key)
                        segment_data[seg_num][attribute] = value
        
        output_text = f"Number of Segments: {segment_count}"
        print(output_text)
        write_to_word(output_text, indent_level=0, alignment=WD_PARAGRAPH_ALIGNMENT.CENTER)

        if data is not None:
            total_voxel_count = f"{np.sum(data >= 0):,}"  # Sum of all voxels in the image
            output_text = f"Total Voxel Count: {total_voxel_count}"
            print(output_text)
            # write_to_word(output_text, indent_level=8, font_color='orange')

        for seg_num, attributes in segment_data.items():
            segment_name_text = f"Segment {seg_num} Name: {attributes.get('Name', 'N/A')} ■"  # Concatenate the '■' character here
            # segment_id = f"Segment {seg_num} ID: {attributes.get('ID', 'N/A')}"
            # segment_color = f"Segment {seg_num} Color: {attributes.get('Color', 'N/A')}"
            segment_label_value = f"LabelValue: {attributes.get('LabelValue', 'N/A')}"

            if attributes.get('Color', 'N/A') != 'N/A':
                color_value = attributes.get('Color', 'N/A')
                r, g, b = [int(float(x) * 255) for x in color_value.split()]

            # Split the segment name write into two: one for the name and one for the '■' with color.
            write_to_word(segment_name_text, indent_level=0, font_color=(r,g,b), alignment=WD_PARAGRAPH_ALIGNMENT.CENTER)  # Write the segment name without '■'
            
            # Add other attributes
            # write_to_word(segment_id, indent_level=10)
            # write_to_word(segment_color, indent_level=0)
            write_to_word(segment_label_value, indent_level=0, alignment=WD_PARAGRAPH_ALIGNMENT.CENTER, highlight='yellow')
            label_value = attributes.get('LabelValue')

            if label_value:
                label_value = int(label_value)  # Ensure it's an integer
                voxel_count = f"{np.sum(data == label_value):,}"  # Count and format with commas
                voxel_count_text = f"Voxel Count: {voxel_count}"
                write_to_word(voxel_count_text, indent_level=0, alignment=WD_PARAGRAPH_ALIGNMENT.CENTER, highlight='yellow')
            write_to_word(" ")

    except Exception as e:
        output_text = f"An error occurred: {str(e)}"
        print(output_text)
        write_to_word(output_text)

#10 Process Nrrd File in folder 
def process_nrrd_files_in_folder(folder_path): 
    add_horizontal_line(doc)
    # Extract case ID from the folder path
    

# Initialize the dictionary to keep track of added images
    pattern = r"Case-\d+"
    match = re.search(pattern, folder_path)
    case_id = match.group()
    files_and_subdirs = os.listdir(folder_path)
    output_text = f"File: {case_id}"
    print(output_text)
    write_to_word(output_text, bold=True, alignment=WD_PARAGRAPH_ALIGNMENT.CENTER, highlight='red')
    # write_to_word(" ")

    not_processed = [item for item in files_and_subdirs if not (item.startswith("S-") and item.endswith('.nrrd'))]
    if not_processed:
        output_text = f"Files NOT being processed: {len(not_processed)}"
        print(output_text)
        write_to_word(output_text, bold=True, indent_level=0, font_color='blue', alignment=WD_PARAGRAPH_ALIGNMENT.CENTER)
        for index, item in enumerate(not_processed, 1): # Start the index from 1
            output_text = f"{index}. {item}" # Replace dashes with numbers
            print(output_text)
            write_to_word(output_text, indent_level=0, font_color='blue', alignment=WD_PARAGRAPH_ALIGNMENT.CENTER)
        write_to_word(" ")
    else:
        output_text = "All files in this folder will be processed."
        print(output_text)
        write_to_word(output_text, bold=True, indent_level=0, font_color='blue', alignment=WD_PARAGRAPH_ALIGNMENT.CENTER)

    to_process = [item for item in files_and_subdirs if item.startswith("S-") and item.endswith('.nrrd')]
    if to_process:
        output_text = f"Files being processed: {len(to_process)}" # Number of items being processed

        print(output_text)
        write_to_word(output_text, bold=True, indent_level=0, font_color='blue', alignment=WD_PARAGRAPH_ALIGNMENT.CENTER)
        
        for index, item in enumerate(to_process, 1): # Start the index from 1
            output_text = f"{index}. {item}" 
            print(output_text)
            # write_to_word(output_text, indent_level=0, font_color='blue')

    for item in files_and_subdirs:
        if item.startswith("S-"):
            if item.startswith("S-") and item.endswith('.nrrd'):
                full_item_path = os.path.join(folder_path, item)
                case_id = extract_case_identifier(full_item_path)
                creation_time = os.path.getctime(full_item_path)
                formatted_creation_time = datetime.fromtimestamp(creation_time).strftime('%Y/%m/%d %H:%M')
                image = sitk.ReadImage(full_item_path)
                data = sitk.GetArrayFromImage(image)

                # Determine the font color
                if "label" in item.lower():
                    font_color_val = 'green'
                else:
                    font_color_val = None  # default color
                output_text = f"{item}"
                output_filename_NIFTI = item.replace("S-", "SS-").replace(".nrrd", ".nii.gz").replace(".seg.",".")
                output_filename_save = "→ ", output_filename_NIFTI
                print(output_text)
                write_to_word(output_text, bold=True, indent_level=0, font_color='white', alignment=WD_PARAGRAPH_ALIGNMENT.CENTER, highlight='blue')
                write_to_word(output_filename_save, bold=True, indent_level=0, font_color='red', alignment=WD_PARAGRAPH_ALIGNMENT.CENTER)
                write_to_word(f"Created on: {formatted_creation_time}", indent_level=0, alignment=WD_PARAGRAPH_ALIGNMENT.CENTER)
                image = sitk.ReadImage(full_item_path)
                data = sitk.GetArrayFromImage(image)
                print_image_details(image, data, full_item_path)
                                         
                if case_id in case_image_map:
                    image_path = case_image_map[case_id]
                    add_image_to_word(image_path, width=1)




                new_case_path, snapshot_directory_of_png = visualize_nrrd_and_save_snapshot(full_item_path, snapshot_directory)
                add_image_to_word(snapshot_directory_of_png, width=1)     
                
                if "label" in item.lower():
                    num_segments, segment_names, label_values, voxel_counts = extract_segments_info_for_labels(full_item_path)
                    if num_segments is not None and num_segments > 0:
                        output_text = f"Number of Segments: {num_segments}"
                        print(output_text)
                        write_to_word(output_text, indent_level=0, font_color=font_color_val, alignment=WD_PARAGRAPH_ALIGNMENT.CENTER)
                       
                        for i in range(num_segments):
                            
                            segment_output = [
                                f"Segment {i + 1}:",
                                f"Label Value: {label_values[i]} \n"
                                f"Voxel Count: {format(voxel_counts[label_values[i]], ',')}"
                            ]
                            for line in segment_output:
                                print(line)
                                write_to_word(line, indent_level=0, font_color=font_color_val, alignment=WD_PARAGRAPH_ALIGNMENT.CENTER, highlight='yellow', label_file=True)
                        
                        # write_to_word(" ")
                else:
                    print_filtered_nrrd_header(image, data)
        add_horizontal_line(doc)
        
#11 Search and Process Files 
def search_and_process(base_path, max_cases=2):
    processed_cases = 0  # Counter to keep track of processed cases
    for root, dirs, files in os.walk(base_path):
        for dir_name in dirs:
            if processed_cases >= max_cases:
                break  # Exit the loop if max_cases is reached
            if dir_name == "1-Duplicates":
                continue
            if dir_name.startswith("Case-"):
                for sub_root, sub_dirs, sub_files in os.walk(os.path.join(root, dir_name)):
                    for sub_dir_name in sub_dirs:
                        if sub_dir_name.startswith("88-"):
                            process_nrrd_files_in_folder(os.path.join(sub_root, sub_dir_name))
                            processed_cases += 1  # Increment the counter
                            if processed_cases >= max_cases:
                                return 

 
#12 Define the directory path and call all functions previously
base_folder_path = "z:/D-Images/"
snapshot_directory = "z:/Q- Figures&Papers/1-T32-Poster/Nate/CaseFileCorrection/"
vx3 = "z:/Q- Figures&Papers/1-T32-Poster/Nate/ImageCreationFile/QC-Images/"
image_files = sorted(os.listdir(vx3))
image_paths = [os.path.join(vx3, file) for file in image_files if file.endswith('.png')]
search_and_process(base_folder_path)

# Save the document
doc.save('z:/Q- Figures&Papers/1-T32-Poster/Nate/CaseFileCorrection/CaseFileCorrection(V14).docx')

File: Case-01
Files NOT being processed: 9
1. 99-Duncan
2. 99-Soheil
3. F-1-QC.mrk.json
4. F-2-ComplicatedPoints.mrk.json
5. F-3.mrk.json
6. F-4-Soheils.mrk.json
7. Trash
8. Vx3-2.nrrd
9. Vx3.nrrd
Files being processed: 10
1. S-Final-1.nrrd
2. S-Final-2.nrrd
3. S-Final-3-label.nrrd
4. S-Final-3.nrrd
5. S-Seeds-1.nrrd
6. S-Segmentation-1.nrrd
7. S-Segmentation-2.nrrd
8. S-Segmentation-3.nrrd
9. S-Segmentation-4.nrrd
10. S-Segmentation-5.nrrd
S-Final-1.nrrd
Dimensions: 625 x 451 x 632
Spacing: 0.38 x 0.38 x 0.38 mm
Snapshot saved to z:/Q- Figures&Papers/1-T32-Poster/Nate/CaseFileCorrection/Case-01-MESAL-5010098-E3\Case-01-MESAL-5010098-E3-S-Final-1.png
Number of Segments: 3
Total Voxel Count: 178,145,000
S-Final-2.nrrd
Dimensions: 625 x 451 x 632
Spacing: 0.38 x 0.38 x 0.38 mm
Snapshot saved to z:/Q- Figures&Papers/1-T32-Poster/Nate/CaseFileCorrection/Case-01-MESAL-5010098-E3\Case-01-MESAL-5010098-E3-S-Final-2.png
Number of Segments: 3
Total Voxel Count: 178,145,000
S-Final-3-label.nrrd


In [32]:
import os
from docx import Document
from docx.shared import Inches
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.oxml import parse_xml
from docx.oxml.ns import nsdecls

def insert_image_into_document(image_folder, document_path):
    # Find the first image in the folder
    for file_name in os.listdir(image_folder):
        if file_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
            image_path = os.path.join(image_folder, file_name)
            break
    else:
        print("No images found in the folder.")
        return

    # Load the Word document
    try:
        doc = Document(document_path)
    except Exception as e:
        print(f"Failed to open the document: {e}")
        return

    # Add a paragraph and align it to center
    paragraph = doc.add_paragraph()
    paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

    # Insert the image
    run = paragraph.add_run()
    run.add_picture(image_path, width=Inches(4))  # Adjust width as needed

    # Adjust the image wrapping style
    last_paragraph = doc.paragraphs[-1] 
    last_run = last_paragraph.runs[-1]
    drawing = last_run.element.xpath('.//wp:inline')[0]
    wrap_in_front = parse_xml(r'<wp:wrapNone {} />'.format(nsdecls('wp')))
    drawing.append(wrap_in_front)

    # Save the modified document
    doc.save(document_path)
    print(f"Image '{file_name}' has been inserted into the document.")

# Paths
image_folder = "z:/Q- Figures&Papers/1-T32-Poster/Nate/ImageCreationFile/QC-Images/"
document_path = 'z:/Q- Figures&Papers/1-T32-Poster/Nate/CaseFileCorrection/CaseFileCorrection(V10).docx'

# Run the function
insert_image_into_document(image_folder, document_path)


Image 'Case-01-QC-Image(2CM).png' has been inserted into the document.


In [5]:
#100 GOLD STANDARD CODE ALL IN ONE

# Import necessary libraries
import os
import SimpleITK as sitk
import numpy as np
from docx import Document
import time
from datetime import datetime
import nrrd
from mayavi import mlab
from docx.shared import Inches, Pt, RGBColor
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
import re
from docx.oxml.ns import qn
from docx.enum.text import WD_COLOR_INDEX
from docx.oxml import OxmlElement

HIGHLIGHT_COLORS = {
    'yellow': WD_COLOR_INDEX.YELLOW,
    'green': WD_COLOR_INDEX.BRIGHT_GREEN,
    'cyan': WD_COLOR_INDEX.TURQUOISE,
    'magenta': WD_COLOR_INDEX.PINK,
    'blue': WD_COLOR_INDEX.BLUE,
    'red': WD_COLOR_INDEX.RED,
    'none': WD_COLOR_INDEX.AUTO  # No highlighting
}

def add_horizontal_line(doc):
    """Adds a horizontal line to the Word document."""
    # Create a new paragraph for the horizontal line
    para = doc.add_paragraph()
    para_format = para.paragraph_format
    para_format.space_after = Pt(0)  # Reduce space after the paragraph
    para_format.space_before = Pt(0)
    p = para._p  # Access the underlying XML element of the paragraph
    pPr = p.get_or_add_pPr()  # Access paragraph properties

    # Create a border element
    pBdr = OxmlElement('w:pBdr')

    # Create a bottom border element
    bottomBdr = OxmlElement('w:bottom')
    bottomBdr.set(qn('w:val'), 'single')
    bottomBdr.set(qn('w:sz'), '4')  # Size of the border
    bottomBdr.set(qn('w:space'), '1')  # Space between border and text
    bottomBdr.set(qn('w:color'), 'auto')  # Border color

    # Add the bottom border to the border element
    pBdr.append(bottomBdr)

    # Add the border element to the paragraph properties
    pPr.append(pBdr)
def set_section_columns(section, num_columns, space_cm, separator=True):
    """ Set up the number of columns in a Word document section.
    
    Args:
    section (docx.section.Section): The section to modify.
    num_columns (int): The number of columns to set.
    space_cm (float): The spacing between columns in centimeters.
    separator (bool): Whether to add a line between columns.
    """
    sectPr = section._sectPr
    cols = sectPr.xpath('./w:cols')[0]
    cols.set(qn('w:num'), str(num_columns))
    cols.set(qn('w:space'), str(int(space_cm * 20)))  # Word space units are 1/20th of a point
    if separator:
        cols.set(qn('w:sep'), '1')
    else:
        cols.set(qn('w:sep'), '0')

COLORS_DICT = {
    'blue': (79, 129, 189),
    'green': (155, 187, 89),
    'red': (250, 0, 0,),
    'orange': (237, 125, 49),
    'white': (255, 255, 255)
    # Add more colors as needed
}

# Initialize a Document object for Word file output
doc = Document()
# doc = Document("z:/Q- Figures&Papers/1-T32-Poster/Nate/CaseFileCorrection/CaseFileCorrection.docx")
num_columns = 3  # or 3 for three columns
space_between_columns_cm = .5  # Adjust the spacing as necessary
set_section_columns(doc.sections[0], num_columns, space_between_columns_cm)


#2 Function to write data to Word document
##############################################################################################################
def write_to_word(text, bold=False, indent_level=0, font_color=None, alignment=None, highlight=None, label_file=False):
    paragraph = doc.add_paragraph()
    paragraph_format = paragraph.paragraph_format
    paragraph_format.space_after = Pt(0)
    paragraph_format.space_before = Pt(0)
    paragraph_format.left_indent = Pt(12 * indent_level)
    if label_file and 'Voxel Count' in text:
        font_color = 'green'

    # Check if the text contains a colon and a highlight color is specified
    if ':' in text and highlight:
        # Split the text into label and values
        label, values = text.split(':', 1)
        label_run = paragraph.add_run(label + ':')
        label_run.font.name = 'Courier New'
        label_run.font.size = Pt(6)
        if font_color:
            # Apply the font color if specified
            r, g, b = COLORS_DICT.get(font_color, (0, 0, 0))
            label_run.font.color.rgb = RGBColor(r, g, b)
        
        # Now create a new run for the values which will be highlighted
        values_run = paragraph.add_run(values)
        values_run.font.name = 'Courier New'
        values_run.font.size = Pt(6)
        if bold:
            values_run.bold = True

        # Set the highlight color for values
        if highlight in HIGHLIGHT_COLORS:
            values_run.font.highlight_color = HIGHLIGHT_COLORS[highlight]
            if HIGHLIGHT_COLORS[highlight] == WD_COLOR_INDEX.BLUE:
                # If highlight is blue, set font color to white for values
                values_run.font.color.rgb = RGBColor(255, 255, 255)
    else:
        # If there's no colon or no highlight, just treat it as a normal run
        run = paragraph.add_run(text)
        run.font.name = 'Courier New'
        run.font.size = Pt(6)
        if bold:
            run.bold = True

        # Set the font color
        if font_color:
            if isinstance(font_color, tuple) and len(font_color) == 3:
                run.font.color.rgb = RGBColor(*font_color)
            else:
                r, g, b = COLORS_DICT.get(font_color, (0, 0, 0))
                run.font.color.rgb = RGBColor(r, g, b)

        # Set the highlight color for the run
        if highlight in HIGHLIGHT_COLORS:
            run.font.highlight_color = HIGHLIGHT_COLORS[highlight]
            if HIGHLIGHT_COLORS[highlight] == WD_COLOR_INDEX.BLUE:
                # If highlight is blue, set font color to white
                run.font.color.rgb = RGBColor(255, 255, 255)

    # Set paragraph alignment if specified
    if alignment:
        paragraph.alignment = alignment

#3
def add_image_to_word(image_path, width=None):
    try:
        # Create a new paragraph for the image
        image_paragraph = doc.add_paragraph()
        image_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

        # Add an image to the Word document
        if width:
            run = image_paragraph.add_run()
            run.add_picture(image_path, width=Inches(width))
        else:
            run = image_paragraph.add_run()
            run.add_picture(image_path)  # Automatically uses the image's size

    except Exception as e:
        print(f"An error occurred while adding the image: {e}")

#4 Function to Get Image Details
def print_image_details(image, data, full_item_path):
    # Extracting size and spacing information
    size = image.GetSize()
    spacing = image.GetSpacing()
    size_output = f"Dimensions: {size[0]} x {size[1]} x {size[2]}"
    spacing_output = f"Spacing: {spacing[0]:.2f} x {spacing[1]:.2f} x {spacing[2]:.2f} mm"
    
    # Output the size and spacing information
    print(size_output)
    print(spacing_output)
    write_to_word(size_output, indent_level=0, alignment=WD_PARAGRAPH_ALIGNMENT.CENTER, highlight='yellow')
    write_to_word(spacing_output, indent_level=0, alignment=WD_PARAGRAPH_ALIGNMENT.CENTER, highlight='yellow')

#5 Extract segment information for labels
def extract_segments_info_for_labels(file_path):
    try:
        image = sitk.ReadImage(file_path)
        data = sitk.GetArrayFromImage(image)
        unique_values = np.unique(data)
        segment_names = [f"Segment_{value}" for value in unique_values]
        label_values = list(unique_values)
        num_segments = len(label_values)
        voxel_counts = {value: np.sum(data == value) for value in unique_values}

        return num_segments, segment_names, label_values, voxel_counts
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        write_to_word(f"An error occurred: {str(e)}")
        return None, None, None
    
#6 Map Color to Label Value
def map_color_to_label(metadata, color):
    # Convert color tuple from 0-1 to 0-255 range for comparison
    color_255 = tuple(int(c * 255) for c in color)
    # Search in metadata for the label with the given color
    for key, value in metadata.items():
        if key.endswith("_Color"):
            # Extract the RGB values and convert to int in range 0-255
            meta_color = tuple(int(float(v) * 255) for v in value.split())
            if meta_color == color_255:
                # Find the label value associated with this color
                label_key = key.replace('_Color', '_LabelValue')
                return int(metadata[label_key])
    return None

#7 Check segment names
def check_segment_names(metadata, case_path):
    segment_names = []
    for key in sorted(metadata.keys()):
        if key.endswith("_Name"):
            segment_names.append(metadata[key])

    # if len(segment_names) > 3:
    #     print(f"Warning: More than three segments found in {case_path}. Skipping file.")
    #     return False

    if len(segment_names) >= 2:
        if not segment_names[0].startswith('A') or not segment_names[1].startswith('V'):
            print(f"Warning: The segments may be named wrong for {case_path}")
    else:
        print(f"Warning: Not enough segments found in {case_path}")

    return True

#8 Visualization
def visualize_nrrd_and_save_snapshot(case_path, snapshot_directory):
    # try:
        avmlabel, metadata = nrrd.read(case_path)

        OP = 1
        DecimationFactor = 2
        use_label_value = case_path.endswith('label.nrrd') or case_path.endswith('Label.nrrd')

        if use_label_value:
            # When using label values, set the segments based on the label value
            Unraveled_A = (avmlabel == 1).astype(int)
            Unraveled_V = (avmlabel == 2).astype(int)
            color_a = (0, 0.592157, 0.807843)  # Assuming these are your default colors
            color_v = (0.847059, 0.396078, 0.309804)
        else:
            if not check_segment_names(metadata, case_path):
                return
            # Define the colors for 'A' and 'V' segments
            color_a = (0, 0.592157, 0.807843)  # Assuming these are your default colors
            color_v = (0.847059, 0.396078, 0.309804)
        # Find the label values that correspond to the colors
            label_a = map_color_to_label(metadata, color_a)
            label_v = map_color_to_label(metadata, color_v)

            Unraveled_A = (avmlabel == label_a).astype(int)
            Unraveled_V = (avmlabel == label_v).astype(int)

        # Create Unraveled seagments for 'A' and 'V'
        
        # Now use 'Unraveled_A' and 'Unraveled_V' to visualize in Mayavi
        figure_name = os.path.basename(case_path).split('.')[0]
        fig = mlab.figure(figure_name, bgcolor=(1, 1, 1))

        if Unraveled_A.any():
            mlab.contour3d(Unraveled_A[::DecimationFactor, ::DecimationFactor, ::DecimationFactor],
                    contours=[0.5], color=color_a, opacity=OP)

        # Visualize the Unraveled_V segment
        if Unraveled_V.any():
            mlab.contour3d(Unraveled_V[::DecimationFactor, ::DecimationFactor, ::DecimationFactor],
                    contours=[0.5], color=color_v, opacity=OP)

        # Camera and snapshot settings
        camera = mlab.gcf().scene.camera
        camera.position = [107, 140, 613]
        camera.focal_point = [175, 128, 148]
        camera.view_up = [0, 1, 0]
        camera.clipping_range = (1, 1000)

        path_parts = case_path.split(os.sep)  # Split the path into parts
        case_index = [i for i, part in enumerate(path_parts) if part.startswith('Case-')][0]  # Find the index of the part that starts with 'Case-'
        case_name = path_parts[case_index]  # Extract the case name

        # Constructing the directory and file names
        case_directory_name = case_name  # Use the case name as the directory name
        final_file_name = f"{case_name}-{path_parts[-1].replace('.nrrd', '.png')}"  # Construct the file name

        # Directory creation and snapshot saving
        case_directory_path = os.path.join(snapshot_directory, case_directory_name)
        if not os.path.exists(case_directory_path):
            os.makedirs(case_directory_path)

        snapshot_path = os.path.join(case_directory_path, final_file_name) 
        mlab.savefig(snapshot_path)
        
        mlab.close()
        print(f"Snapshot saved to {snapshot_path}")
        return case_path, snapshot_path 

#9 Print Filtered Nrrd Header
def print_filtered_nrrd_header(image, data=None):
    try:
        keys = image.GetMetaDataKeys()
        segment_count = 0
        segment_data = {}
        
        for key in keys:
            if "Segment" in key:
                segment_info = key.split('Segment')[-1]
                if segment_info[0].isdigit():
                    seg_num, attribute = segment_info.split('_', 1)
                    if seg_num not in segment_data:
                        segment_data[seg_num] = {}
                        segment_count += 1
                    if attribute in ['Color', 'ID', 'LabelValue', 'Name']:
                        value = image.GetMetaData(key)
                        segment_data[seg_num][attribute] = value
        
        output_text = f"Number of Segments: {segment_count}"
        print(output_text)
        write_to_word(output_text, indent_level=0, alignment=WD_PARAGRAPH_ALIGNMENT.CENTER)

        if data is not None:
            total_voxel_count = f"{np.sum(data >= 0):,}"  # Sum of all voxels in the image
            output_text = f"Total Voxel Count: {total_voxel_count}"
            print(output_text)
            # write_to_word(output_text, indent_level=8, font_color='orange')

        for seg_num, attributes in segment_data.items():
            segment_name_text = f"Segment {seg_num} Name: {attributes.get('Name', 'N/A')} ■"  # Concatenate the '■' character here
            # segment_id = f"Segment {seg_num} ID: {attributes.get('ID', 'N/A')}"
            # segment_color = f"Segment {seg_num} Color: {attributes.get('Color', 'N/A')}"
            segment_label_value = f"LabelValue: {attributes.get('LabelValue', 'N/A')}"

            if attributes.get('Color', 'N/A') != 'N/A':
                color_value = attributes.get('Color', 'N/A')
                r, g, b = [int(float(x) * 255) for x in color_value.split()]

            # Split the segment name write into two: one for the name and one for the '■' with color.
            write_to_word(segment_name_text, indent_level=0, font_color=(r,g,b), alignment=WD_PARAGRAPH_ALIGNMENT.CENTER)  # Write the segment name without '■'
            
            # Add other attributes
            # write_to_word(segment_id, indent_level=10)
            # write_to_word(segment_color, indent_level=0)
            write_to_word(segment_label_value, indent_level=0, alignment=WD_PARAGRAPH_ALIGNMENT.CENTER, highlight='yellow')
            label_value = attributes.get('LabelValue')

            if label_value:
                label_value = int(label_value)  # Ensure it's an integer
                voxel_count = f"{np.sum(data == label_value):,}"  # Count and format with commas
                voxel_count_text = f"Voxel Count: {voxel_count}"
                write_to_word(voxel_count_text, indent_level=0, alignment=WD_PARAGRAPH_ALIGNMENT.CENTER, highlight='yellow')
            write_to_word(" ")

    except Exception as e:
        output_text = f"An error occurred: {str(e)}"
        print(output_text)
        write_to_word(output_text)

#10 Process Nrrd File in folder 
def process_nrrd_files_in_folder(folder_path):
    add_horizontal_line(doc)
    pattern = r"Case-\d+"
    match = re.search(pattern, folder_path)
    case_id = match.group()
    files_and_subdirs = os.listdir(folder_path)
    output_text = f"File: {case_id}"
    print(output_text)
    write_to_word(output_text, bold=True, alignment=WD_PARAGRAPH_ALIGNMENT.CENTER, highlight='red')
    # write_to_word(" ")

    not_processed = [item for item in files_and_subdirs if not (item.startswith("S-") and item.endswith('.nrrd'))]
    if not_processed:
        output_text = f"Files NOT being processed: {len(not_processed)}"
        print(output_text)
        write_to_word(output_text, bold=True, indent_level=0, font_color='blue', alignment=WD_PARAGRAPH_ALIGNMENT.CENTER)
        for index, item in enumerate(not_processed, 1): # Start the index from 1
            output_text = f"{index}. {item}" # Replace dashes with numbers
            print(output_text)
            write_to_word(output_text, indent_level=0, font_color='blue', alignment=WD_PARAGRAPH_ALIGNMENT.CENTER)
        write_to_word(" ")
    else:
        output_text = "All files in this folder will be processed."
        print(output_text)
        write_to_word(output_text, bold=True, indent_level=0, font_color='blue', alignment=WD_PARAGRAPH_ALIGNMENT.CENTER)

    to_process = [item for item in files_and_subdirs if item.startswith("S-") and item.endswith('.nrrd')]
    if to_process:
        output_text = f"Files being processed: {len(to_process)}" # Number of items being processed
        print(output_text)
        write_to_word(output_text, bold=True, indent_level=0, font_color='blue', alignment=WD_PARAGRAPH_ALIGNMENT.CENTER)
        
        for index, item in enumerate(to_process, 1): # Start the index from 1
            output_text = f"{index}. {item}" 
            print(output_text)
            # write_to_word(output_text, indent_level=0, font_color='blue')

    for item in files_and_subdirs:
        if item.startswith("S-"):
            full_item_path = os.path.join(folder_path, item)
            if os.path.isfile(full_item_path) and item.endswith('.nrrd'):
                creation_time = os.path.getctime(full_item_path)
                formatted_creation_time = datetime.fromtimestamp(creation_time).strftime('%Y/%m/%d %H:%M')
                image = sitk.ReadImage(full_item_path)
                data = sitk.GetArrayFromImage(image)

                # Determine the font color
                if "label" in item.lower():
                    font_color_val = 'green'
                else:
                    font_color_val = None  # default color
                output_text = f"{item}"
                output_filename_NIFTI = item.replace("S-", "SS-").replace(".nrrd", ".nii.gz").replace(".seg.",".")
                output_filename_save = "→ ", output_filename_NIFTI
                print(output_text)
                write_to_word(output_text, bold=True, indent_level=0, font_color='white', alignment=WD_PARAGRAPH_ALIGNMENT.CENTER, highlight='blue')
                write_to_word(output_filename_save, bold=True, indent_level=0, font_color='red', alignment=WD_PARAGRAPH_ALIGNMENT.CENTER)
                write_to_word(f"Created on: {formatted_creation_time}", indent_level=0, alignment=WD_PARAGRAPH_ALIGNMENT.CENTER)
                image = sitk.ReadImage(full_item_path)
                data = sitk.GetArrayFromImage(image)
                print_image_details(image, data, full_item_path)
                # write_to_word(' ')
                
                new_case_path, snapshot_directory_of_png = visualize_nrrd_and_save_snapshot(full_item_path, snapshot_directory)
                add_image_to_word(snapshot_directory_of_png, width=1)     

                if "label" in item.lower():
                    num_segments, segment_names, label_values, voxel_counts = extract_segments_info_for_labels(full_item_path)
                    if num_segments is not None and num_segments > 0:
                        output_text = f"Number of Segments: {num_segments}"
                        print(output_text)
                        write_to_word(output_text, indent_level=0, font_color=font_color_val, alignment=WD_PARAGRAPH_ALIGNMENT.CENTER)
                       
                        for i in range(num_segments):
                            
                            segment_output = [
                                f"Segment {i + 1}:",
                                f"Label Value: {label_values[i]} \n"
                                f"Voxel Count: {format(voxel_counts[label_values[i]], ',')}"
                            ]
                            for line in segment_output:
                                print(line)
                                write_to_word(line, indent_level=0, font_color=font_color_val, alignment=WD_PARAGRAPH_ALIGNMENT.CENTER, highlight='yellow', label_file=True)
                        
                        # write_to_word(" ")
                else:
                    print_filtered_nrrd_header(image, data)
        add_horizontal_line(doc)

#11 Search and Process Files 
def search_and_process(base_path, max_cases=2):
    processed_cases = 0  # Counter to keep track of processed cases
    for root, dirs, files in os.walk(base_path):
        for dir_name in dirs:
            if processed_cases >= max_cases:
                break  # Exit the loop if max_cases is reached
            if dir_name == "1-Duplicates":
                continue
            if dir_name.startswith("Case-"):
                for sub_root, sub_dirs, sub_files in os.walk(os.path.join(root, dir_name)):
                    for sub_dir_name in sub_dirs:
                        if sub_dir_name.startswith("88-"):
                            process_nrrd_files_in_folder(os.path.join(sub_root, sub_dir_name))
                            processed_cases += 1  # Increment the counter
                            if processed_cases >= max_cases:
                                return 

        
#12 Define the directory path and call all functions previously
base_folder_path = "z:/D-Images/"
snapshot_directory = "z:/Q- Figures&Papers/1-T32-Poster/Nate/CaseFileCorrection/"
search_and_process(base_folder_path)

# Save the document
doc.save('z:/Q- Figures&Papers/1-T32-Poster/Nate/CaseFileCorrection/CaseFileCorrection(V10).docx')

File: Case-01
Files NOT being processed: 9
1. 99-Duncan
2. 99-Soheil
3. F-1-QC.mrk.json
4. F-2-ComplicatedPoints.mrk.json
5. F-3.mrk.json
6. F-4-Soheils.mrk.json
7. Trash
8. Vx3-2.nrrd
9. Vx3.nrrd
Files being processed: 10
1. S-Final-1.nrrd
2. S-Final-2.nrrd
3. S-Final-3-label.nrrd
4. S-Final-3.nrrd
5. S-Seeds-1.nrrd
6. S-Segmentation-1.nrrd
7. S-Segmentation-2.nrrd
8. S-Segmentation-3.nrrd
9. S-Segmentation-4.nrrd
10. S-Segmentation-5.nrrd
S-Final-1.nrrd
Dimensions: 625 x 451 x 632
Spacing: 0.38 x 0.38 x 0.38 mm
Snapshot saved to z:/Q- Figures&Papers/1-T32-Poster/Nate/CaseFileCorrection/Case-01-MESAL-5010098-E3\Case-01-MESAL-5010098-E3-S-Final-1.png
Number of Segments: 3
Total Voxel Count: 178,145,000
S-Final-2.nrrd
Dimensions: 625 x 451 x 632
Spacing: 0.38 x 0.38 x 0.38 mm
Snapshot saved to z:/Q- Figures&Papers/1-T32-Poster/Nate/CaseFileCorrection/Case-01-MESAL-5010098-E3\Case-01-MESAL-5010098-E3-S-Final-2.png
Number of Segments: 3
Total Voxel Count: 178,145,000
S-Final-3-label.nrrd


In [ ]:
#12 Define the directory path and call all functions previously
base_folder_path = "z:/D-Images/"
snapshot_directory = "z:/Q- Figures&Papers/1-T32-Poster/Nate/CaseFileCorrection/"
search_and_process(base_folder_path)

# Save the document
doc.save('z:/Q- Figures&Papers/1-T32-Poster/Nate/CaseFileCorrection/CaseFileCorrection.docx')

In [3]:
import os
import re

def get_file_number(file_name):
    # Extract the number from the filename using regex
    match = re.search(r'\d+', file_name)
    return int(match.group()) if match else None

def rename_files_in_directory(directory_path):
    # List all files in the directory
    all_files = os.listdir(directory_path)
    
    # Filter out the files that match the 'S-' prefix and '.nrrd' suffix
    s_files = [f for f in all_files if f.startswith('S-') and f.endswith('.nrrd')]
    
    # Separate the files into their respective categories
    seeds_files = sorted([f for f in s_files if 'S-Seeds' in f], key=get_file_number)
    segmentation_files = sorted([f for f in s_files if 'S-Segmentation' in f], key=get_file_number)
    final_files = sorted([f for f in s_files if 'S-Final-' in f and '-label' not in f], key=get_file_number)
    
    # Identify and remove the label files
    label_files = [f for f in s_files if '-label.nrrd' in f]
    for label_file in label_files:
        os.remove(os.path.join(directory_path, label_file))
        print(f'Removed label file: {label_file}')
    
    # Rename 'S-Seeds-N.nrrd' files
    for i, file_name in enumerate(seeds_files, start=1):
        new_name = f"S-Seeds-{i}.nrrd"
        print(f'Renaming "{file_name}" to "{new_name}"')
        os.rename(os.path.join(directory_path, file_name), os.path.join(directory_path, new_name))

    # Rename 'S-Segmentation-N.nrrd' files
    for i, file_name in enumerate(segmentation_files, start=1):
        new_name = f"S-Segmentation-{i}.nrrd"
        print(f'Renaming "{file_name}" to "{new_name}"')
        os.rename(os.path.join(directory_path, file_name), os.path.join(directory_path, new_name))

    # Rename 'S-Final-N.nrrd' files into 'S-Segmentation' series
    if final_files:  # Check if there are any final files
        # Rename all but the last 'S-Final' file into the 'S-Segmentation' series
        for file_name in final_files[:-1]:  # Skip the last file
            next_seg_num = len(segmentation_files) + 1
            new_name = f"S-Segmentation-{next_seg_num}.nrrd"
            print(f'Renaming "{file_name}" to "{new_name}"')
            os.rename(os.path.join(directory_path, file_name), os.path.join(directory_path, new_name))
            segmentation_files.append(new_name)  # Add new name to the list to continue the sequence
        
        # Rename the last 'S-Final' file to just 'S-Final.nrrd'
        highest_final_file = final_files[-1]
        new_final_name = "S-Final.nrrd"
        print(f'Renaming "{highest_final_file}" to "{new_final_name}"')
        os.rename(os.path.join(directory_path, highest_final_file), os.path.join(directory_path, new_final_name))

    print("Renaming complete.")

# Example usage:
directory_path = 'z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-53-Spiromics-46195695/Markups/91-Nate/'
rename_files_in_directory(directory_path)

# This code is ready to be tested in your environment. 
# Remember to set the 'directory_path' variable to the path of your testing directory before running.



Renaming "S-Seeds-1.nrrd" to "S-Seeds-1.nrrd"
Renaming "S-Seeds-2.nrrd" to "S-Seeds-2.nrrd"
Renaming "S-Segmentation-1.nrrd" to "S-Segmentation-1.nrrd"
Renaming "S-Segmentation-2.nrrd" to "S-Segmentation-2.nrrd"
Renaming "S-Segmentation-3.nrrd" to "S-Segmentation-3.nrrd"
Renaming "S-Segmentation-5.nrrd" to "S-Segmentation-4.nrrd"
Renaming "S-Segmentation-50.nrrd" to "S-Segmentation-5.nrrd"
Renaming "S-Segmentation-61.nrrd" to "S-Segmentation-6.nrrd"
Renaming "S-Final-8.nrrd" to "S-Segmentation-7.nrrd"
Renaming "S-Final-91.nrrd" to "S-Final.nrrd"
Renaming complete.


In [ ]:
#1 Golden STANDARD NEED TO ADD VISUALIZATION AND PRINT TO WORD DOCUMENT 
# Import necessary libraries
import os
import SimpleITK as sitk
import numpy as np
from docx import Document
from docx.shared import RGBColor
from docx.shared import Pt
import time
from datetime import datetime


COLORS_DICT = {
    'blue': (79, 129, 189),
    'green': (155, 187, 89),
    'red': (250, 0, 0,),
    'orange': (237, 125, 49)
    # Add more colors as needed
}

# Initialize a Document object for Word file output
doc = Document()


#2 Function to write data to Word document
def write_to_word(text, bold=False, indent_level=0, font_color=None):
    paragraph = doc.add_paragraph(text)
    paragraph_format = paragraph.paragraph_format
    paragraph_format.space_after = Pt(0)
    paragraph_format.space_before = Pt(0)
    paragraph_format.left_indent = Pt(12 * indent_level)
    run = paragraph.runs[0]
    if bold:
        run.bold = True
    font = run.font
    font.name = 'Courier New'
    font.size = Pt(12)
    if font_color:
        # Check if the font_color is a named color
        if isinstance(font_color, str) and font_color in COLORS_DICT:
            r, g, b = COLORS_DICT[font_color]
        else:
            r, g, b = font_color
        font.color.rgb = RGBColor(r, g, b)
  
#3
def print_image_details(image, data, full_item_path):
    # Extracting size and spacing information
    size = image.GetSize()
    spacing = image.GetSpacing()
    size_output = f"Dimensions: {size[0]} x {size[1]} x {size[2]}"
    spacing_output = f"Spacing: {spacing[0]:.2f} x {spacing[1]:.2f} x {spacing[2]:.2f} mm"
    
    # Output the size and spacing information
    print(size_output)
    print(spacing_output)
    write_to_word(size_output, indent_level=5, font_color='orange')
    write_to_word(spacing_output, indent_level=5, font_color='orange')

#4
def extract_segments_info_for_labels(file_path):
    try:
        image = sitk.ReadImage(file_path)
        data = sitk.GetArrayFromImage(image)
        unique_values = np.unique(data)
        segment_names = [f"Segment_{value}" for value in unique_values]
        label_values = list(unique_values)
        num_segments = len(label_values)
        voxel_counts = {value: np.sum(data == value) for value in unique_values}

        return num_segments, segment_names, label_values, voxel_counts
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        write_to_word(f"An error occurred: {str(e)}")
        return None, None, None

#5
def print_filtered_nrrd_header(image, data=None):
    try:
        keys = image.GetMetaDataKeys()
        segment_count = 0
        segment_data = {}
        
        for key in keys:
            if "Segment" in key:
                segment_info = key.split('Segment')[-1]
                if segment_info[0].isdigit():
                    seg_num, attribute = segment_info.split('_', 1)
                    if seg_num not in segment_data:
                        segment_data[seg_num] = {}
                        segment_count += 1
                    if attribute in ['Color', 'ID', 'LabelValue', 'Name']:
                        value = image.GetMetaData(key)
                        segment_data[seg_num][attribute] = value
        
        output_text = f"Number of Segments: {segment_count}"
        print(output_text)
        write_to_word(output_text, indent_level=8, font_color='orange')

        if data is not None:
            total_voxel_count = f"{np.sum(data >= 0):,}"  # Sum of all voxels in the image
            output_text = f"Total Voxel Count: {total_voxel_count}"
            print(output_text)
            write_to_word(output_text, indent_level=8, font_color='orange')

        for seg_num, attributes in segment_data.items():
            segment_name_text = f"Segment {seg_num} Name: {attributes.get('Name', 'N/A')} ■"  # Concatenate the '■' character here
            segment_id = f"Segment {seg_num} ID: {attributes.get('ID', 'N/A')}"
            segment_color = f"Segment {seg_num} Color: {attributes.get('Color', 'N/A')}"
            segment_label_value = f"Segment {seg_num} LabelValue: {attributes.get('LabelValue', 'N/A')}"

            if attributes.get('Color', 'N/A') != 'N/A':
                color_value = attributes.get('Color', 'N/A')
                r, g, b = [int(float(x) * 255) for x in color_value.split()]

            # Split the segment name write into two: one for the name and one for the '■' with color.
            write_to_word(segment_name_text, indent_level=10, font_color=(r,g,b))  # Write the segment name without '■'
            
            # Add other attributes
            write_to_word(segment_id, indent_level=10)
            write_to_word(segment_color, indent_level=10)
            write_to_word(segment_label_value, indent_level=10)
            label_value = attributes.get('LabelValue')

            if label_value:
                label_value = int(label_value)  # Ensure it's an integer
                voxel_count = f"{np.sum(data == label_value):,}"  # Count and format with commas
                voxel_count_text = f"Segment {seg_num} Voxel Count: {voxel_count}"
                write_to_word(voxel_count_text, indent_level=10)

            write_to_word(" ")
    except Exception as e:
        output_text = f"An error occurred: {str(e)}"
        print(output_text)
        write_to_word(output_text)

def process_nrrd_files_in_folder(folder_path):

    files_and_subdirs = os.listdir(folder_path)
    output_text = f"Folder Path: {folder_path}"
    print(output_text)
    write_to_word(output_text, bold=True)
    write_to_word(" ")

    not_processed = [item for item in files_and_subdirs if not (item.startswith("S-") and item.endswith('.nrrd'))]
    if not_processed:
        output_text = f"Files/Folders NOT being processed: {len(not_processed)}"
        print(output_text)
        write_to_word(output_text, bold=True, indent_level=5, font_color='blue')
        for index, item in enumerate(not_processed, 1): # Start the index from 1
            output_text = f"{index}. {item}" # Replace dashes with numbers
            print(output_text)
            write_to_word(output_text, indent_level=8, font_color='blue')
        write_to_word(" ")
    else:
        output_text = "All files in this folder will be processed."
        print(output_text)
        write_to_word(output_text, bold=True, indent_level=10, font_color='blue')

    to_process = [item for item in files_and_subdirs if item.startswith("S-") and item.endswith('.nrrd')]
    if to_process:
        output_text = f"Files being processed: {len(to_process)}" # Number of items being processed
        print(output_text)
        write_to_word(output_text, bold=True, indent_level=5, font_color='blue')
        
        for index, item in enumerate(to_process, 1): # Start the index from 1
            output_text = f"{index}. {item}" 
            print(output_text)
            write_to_word(output_text, indent_level=8, font_color='blue')
        write_to_word(" ")

    for item in files_and_subdirs:
        if item.startswith("S-"):
            full_item_path = os.path.join(folder_path, item)
            if os.path.isfile(full_item_path) and item.endswith('.nrrd'):
                creation_time = os.path.getctime(full_item_path)
                formatted_creation_time = datetime.fromtimestamp(creation_time).strftime('%Y/%m/%d %H:%M')
                image = sitk.ReadImage(full_item_path)
                data = sitk.GetArrayFromImage(image)

                # Determine the font color
                if "label" in item.lower():
                    font_color_val = 'green'
                else:
                    font_color_val = None  # default color
                output_text = f"Processing: {item}"
                print(output_text)
                write_to_word(output_text, bold=True, indent_level=5)
                write_to_word(f"Created on: {formatted_creation_time}", indent_level=5, font_color='orange')
                image = sitk.ReadImage(full_item_path)
                data = sitk.GetArrayFromImage(image)
                print_image_details(image, data, full_item_path)
                write_to_word(' ')
                
                # MAking the 3d and 
                RaveledA, RaveledV = UnraveledAVAs01(SegImage)
                import mayavi
                mlab.figure()
                mlab.contour3d(RaveledA, contour = [0.5], blue)
                mlab.contour3d(Raveledv, contour = [0.5], red)
                mlab.savefig()
                write_to_word.image(snapshot locatoion)

                # Placing the coronal snapshot with segments overlaid









                if "label" in item.lower():
                    num_segments, segment_names, label_values, voxel_counts = extract_segments_info_for_labels(full_item_path)
                    if num_segments is not None and num_segments > 0:
                        output_text = f"Number of Segments: {num_segments}"
                        print(output_text)
                        write_to_word(output_text, indent_level=8, font_color=font_color_val)
                        for i in range(num_segments):
                            segment_output = [
                                f"Segment {i + 1}:",
                                f"  Name: {segment_names[i]}",
                                f"  Label Value: {label_values[i]}"
                                f"  Voxel Count: {format(voxel_counts[label_values[i]], ',')}"
                            ]
                            for line in segment_output:
                                print(line)
                                write_to_word(line, indent_level=10, font_color=font_color_val)
                        write_to_word(" ")

                            
                else:
                    print_filtered_nrrd_header(image, data)

def search_and_process(base_path, max_cases=2):
    processed_cases = 0  # Counter to keep track of processed cases
    
    for root, dirs, files in os.walk(base_path):
        for dir_name in dirs:
            if processed_cases >= max_cases:
                break  # Exit the loop if max_cases is reached
            if dir_name == "1-Duplicates":
                continue
            if dir_name.startswith("Case-"):
                for sub_root, sub_dirs, sub_files in os.walk(os.path.join(root, dir_name)):
                    for sub_dir_name in sub_dirs:
                        if sub_dir_name.startswith("88-"):
                            process_nrrd_files_in_folder(os.path.join(sub_root, sub_dir_name))
                            processed_cases += 1  # Increment the counter



# Define the directory path
base_folder_path = "z:/D-Images"
search_and_process(base_folder_path)

# Save the document
doc.save('z:/Q- Figures&Papers/1-T32-Poster/Nate/CaseFileCorrection/CaseFileCorrection.docx')


In [ ]:
# GOLD STANDARD TO DISPLAY MAYAVI VIEWERS WORKS WITH NRRD

from mayavi import mlab
import numpy as np
import nrrd

def update_camera_info(obj, evt):
    camera = mlab.gcf().scene.camera
    print("Camera Position:", camera.position)
    print("Camera Focal Point:", camera.focal_point)
    print("Camera View Up:", camera.view_up)

LabelPath = "z:/D-Images/MESAL-3/2-Checked-Done-Temp/Done/Case-88-MESAL-6010377-E3/Markups/88-Adam/S-Final-1-label.nrrd"

avmlabel, metadata = nrrd.read(LabelPath)


OP = 1
DecimationFactor = 2

figure_name = LabelPath.split('/')[-4]  # Assuming '/' is the path separator

# Create a figure with a specific name and size
fig = mlab.figure(figure_name, bgcolor=(1, 1, 1))

obj = (avmlabel == 1).astype(int)
mlab.contour3d(obj[::DecimationFactor,::DecimationFactor,::DecimationFactor], contours=[0.5], color=(0      , 151/255, 206/255), opacity=OP)
obj = (avmlabel == 2).astype(int)
mlab.contour3d(obj[::DecimationFactor,::DecimationFactor,::DecimationFactor], contours=[0.5], color=(216/255, 101/255,  79/255), opacity=OP)

camera = mlab.gcf().scene.camera
mlab.gcf().scene.camera.position = [107, 128, 613]
mlab.gcf().scene.camera.focal_point = [149, 128, 148]
mlab.gcf().scene.camera.view_up = [0, 1, 0]
mlab.gcf().scene.interactor.add_observer('MouseMoveEvent', update_camera_info)
mlab.show()


In [105]:
def extract_case_folder(case_path):
    case_folder_pattern = re.compile(r'Case-\d+')
    match = case_folder_pattern.search(case_path)
    if match:
        return match.group()
    else:
        raise ValueError("No valid 'Case-N' folder name found in the path.")
    

    ### NEED TO UPDATE THIS TO FIND SEGMENTATION NAME OF ITEM IN CODE BLOCKS ABOVE


In [ ]:
# import nrrd

# # Define the path to your .nrrd file
# file_path = 'z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-76-Spiromics-52980793/Markups/88-Nate/S-Seeds-1.seg.nrrd'

# # Use the nrrd.read function to read the file
# data, header = nrrd.read(file_path)

# # Print the header information vertically
# for key, value in header.items():
#     print(f"{key}: {value}")

import nrrd
import numpy as np

# Define the path to your .nrrd file
file_path = 'z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-76-Spiromics-52980793/Markups/88-Nate/S-Seeds-1.seg.nrrd'

# Use the nrrd.read function to read the file
data, header = nrrd.read(file_path)

# Print the header information
for key, value in header.items():
    print(f"{key}: {value}")

# Count the number of voxels in each segment
# Assuming each unique non-zero value in the data array represents a different segment
unique_values, counts = np.unique(data, return_counts=True)

# Display voxel counts for each segment
for value, count in zip(unique_values, counts):
    if value != 0:  # Assuming 0 is the background or non-segmented area
        print(f"Segment {value}: {count} voxels")


In [ ]:
import nibabel as nib

# Define the path to your .nii.gz file
file_path = 'z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-29-Spiromics-47117390/Markups/88-Adam/(NATE-DELETE)S-Final-2-label.nii.gz'
# Use the nib.load function to read the file
img = nib.load(file_path)

# The 'img' variable now contains the image data and header information
header = img.header

# Print the header information
print(header)

In [54]:
import SimpleITK as sitk
import numpy as np

def remove_segment_and_save_nifti(input_path, output_path, label_value_of_interest):
    # Load the segmentation file
    seg = sitk.ReadImage(input_path)

    # Convert the segmentation image to a numpy array
    seg_array = sitk.GetArrayFromImage(seg)

    # Remove the segment with the specified label value
    seg_array[seg_array == label_value_of_interest] = 0

    # Convert the numpy array back to a SimpleITK image
    seg_modified = sitk.GetImageFromArray(seg_array)
    seg_modified.CopyInformation(seg)  # Preserve the original image's metadata

    # Save the modified segmentation as a NIfTI file
    sitk.WriteImage(seg_modified, output_path, True)  # The 'True' parameter enables compression


input_segmentation_path = 'z:/Q- Figures&Papers/1-T32-Poster/Nate/CaseFileCorrection/Case-01/Misc-Segs/S-Final-1.nrrd'  # Replace with your file's path
output_nifti_path = 'z:/Q- Figures&Papers/1-T32-Poster/Nate/CaseFileCorrection/Case-01/Misc-Segs/S-Final-1.nii.gz'     # Replace with your desired output file path
label_value_to_remove = 3  #(MISC VALUE)                       # Replace with the label value of the segment to remove

remove_segment_and_save_nifti(input_segmentation_path, output_nifti_path, label_value_to_remove)


In [ ]:
## For Nifti Files

from mayavi import mlab
import numpy as np
import nibabel as nib
import os
import re
import nrrd
import copy

# CasePath = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-29-Spiromics-47117390/Markups/88-Adam/S-Seeds-1.nrrd"
CasePath = "z:/D-Images/MESAL-3/2-Checked-Done-Temp/Done/Case-01-MESAL-5010098-E3/Markups/88-Nate/S-Final-1.nii.gz"
filename = os.path.basename(CasePath)

nifti_data = nib.load(CasePath)
avmlabel = nifti_data.get_fdata()

# avmlabel, metadata = nrrd.read(CasePath)
filename_ext = os.path.splitext(filename)[0]

OP = 1
DecimationFactor = 2

# Initialize case_folder before attempting to extract it
case_folder_pattern = re.compile(r'Case-\d+')
match = case_folder_pattern.search(CasePath + filename)

if match:
    case_folder = match.group()
else:
    raise ValueError("No valid 'Case-N' folder name found in the path.")

# Output the extracted case folder for debugging
print(f"Extracted case folder: {case_folder}")

figure_name = case_folder  # Use extracted case folder as the figure name

# Create a figure with a specific name and size
fig = mlab.figure(figure_name, bgcolor=(1, 1, 1))


avmlabel_plot = copy.deepcopy(avmlabel)
avmlabelmax = avmlabel
avmlabelmin = avmlabel
print(avmlabelmax)
print(avmlabelmin)
# avm_label_plot = 

# if sum(avmlabel==1).astype(int):                         
obj = (avmlabel == 1).astype(int) #(ASEED LABEL VALUE)
mlab.contour3d(obj[::DecimationFactor, ::DecimationFactor, ::DecimationFactor], contours=[0.5], color=(0, 151/255, 206/255), opacity=OP)
obj = (avmlabel == 4).astype(int) #(VSEED LABEL VALUE)
mlab.contour3d(obj[::DecimationFactor, ::DecimationFactor, ::DecimationFactor], contours=[0.5], color=(216/255, 101/255,  79/255), opacity=OP)
# obj = (avmlabel == 3).astype(int)
# mlab.contour3d(obj[::DecimationFactor, ::DecimationFactor, ::DecimationFactor], contours=[0.5], color=(216/255, 200/255,  1/255), opacity=.8)
# obj = (avmlabel == 4).astype(int)
# mlab.contour3d(obj[::DecimationFactor, ::DecimationFactor, ::DecimationFactor], contours=[0.5], color=(216/255, 200/255,  1/255), opacity=.8)




camera = mlab.gcf().scene.camera
mlab.gcf().scene.camera.position = [107, 140, 613]
mlab.gcf().scene.camera.focal_point = [149, 128, 148]
mlab.gcf().scene.camera.view_up = [0, 1, 0]
camera.clipping_range = (1, 1000)

snapshot_directory = "z:/Q- Figures&Papers/1-T32-Poster/Nate/CaseFileCorrection/"

# Combine the snapshot directory with the new case folder to create the directory path
case_directory_path = os.path.join(snapshot_directory, case_folder)

# Check if the directory exists, and if not, create it
if not os.path.exists(case_directory_path):
    os.makedirs(case_directory_path)

# Combine the case directory path with the figure name to create the snapshot path
snapshot_path = os.path.join(case_directory_path, f"{figure_name}-{filename_ext}.png")

# Save a snapshot of the Mayavi figure
mlab.savefig(snapshot_path)

# Close the Mayavi figure
mlab.close()

# Optionally, you can display a message indicating that the snapshot has been saved
print(f"Snapshot saved to {snapshot_path}")
print(f"Filename: {filename}")



In [ ]:
# FOR .nrrd files

from mayavi import mlab
import numpy as np
import nibabel as nib
import os
import re
import nrrd
import copy

# CasePath = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-29-Spiromics-47117390/Markups/88-Adam/S-Seeds-1.nrrd"
CasePath = "z:/D-Images/MESAL-3/2-Checked-Done-Temp/Done/Case-01-MESAL-5010098-E3/Markups/88-Nate/S-Segmentation-6.seg.nrrd"
filename = os.path.basename(CasePath)


avmlabel, metadata = nrrd.read(CasePath)
filename_ext = os.path.splitext(filename)[0]

OP = 1
DecimationFactor = 2

# Initialize case_folder before attempting to extract it
case_folder_pattern = re.compile(r'Case-\d+')
match = case_folder_pattern.search(CasePath + filename)

if match:
    case_folder = match.group()
else:
    raise ValueError("No valid 'Case-N' folder name found in the path.")

# Output the extracted case folder for debugging
print(f"Extracted case folder: {case_folder}")

figure_name = case_folder  # Use extracted case folder as the figure name

# Create a figure with a specific name and size
fig = mlab.figure(figure_name, bgcolor=(1, 1, 1))


avmlabel_plot = copy.deepcopy(avmlabel)
avmlabelmax = avmlabel
avmlabelmin = avmlabel
print(avmlabelmax)
print(avmlabelmin)
# avm_label_plot = 

# if sum(avmlabel==1).astype(int):                         
obj = (avmlabel == 1).astype(int)
mlab.contour3d(obj[::DecimationFactor, ::DecimationFactor, ::DecimationFactor], contours=[0.5], color=(0, 151/255, 206/255), opacity=OP)
obj = (avmlabel == 2).astype(int)
mlab.contour3d(obj[::DecimationFactor, ::DecimationFactor, ::DecimationFactor], contours=[0.5], color=(216/255, 101/255,  79/255), opacity=OP)
# obj = (avmlabel == 3).astype(int)
# mlab.contour3d(obj[::DecimationFactor, ::DecimationFactor, ::DecimationFactor], contours=[0.5], color=(216/255, 200/255,  1/255), opacity=.8)
# obj = (avmlabel == 4).astype(int)
# mlab.contour3d(obj[::DecimationFactor, ::DecimationFactor, ::DecimationFactor], contours=[0.5], color=(216/255, 200/255,  1/255), opacity=.8)




camera = mlab.gcf().scene.camera
mlab.gcf().scene.camera.position = [107, 140, 613]
mlab.gcf().scene.camera.focal_point = [149, 128, 148]
mlab.gcf().scene.camera.view_up = [0, 1, 0]
camera.clipping_range = (1, 1000)

snapshot_directory = "z:/Q- Figures&Papers/1-T32-Poster/Nate/CaseFileCorrection/"

# Combine the snapshot directory with the new case folder to create the directory path
case_directory_path = os.path.join(snapshot_directory, case_folder)

# Check if the directory exists, and if not, create it
if not os.path.exists(case_directory_path):
    os.makedirs(case_directory_path)

# Combine the case directory path with the figure name to create the snapshot path
snapshot_path = os.path.join(case_directory_path, f"{figure_name}-{filename_ext}.png")

# Save a snapshot of the Mayavi figure
mlab.savefig(snapshot_path)

# Close the Mayavi figure
mlab.close()

# Optionally, you can display a message indicating that the snapshot has been saved
print(f"Snapshot saved to {snapshot_path}")
print(f"Filename: {filename}")



In [ ]:
from mayavi import mlab
import numpy as np
import nibabel as nib
import os
import re

CasePath = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-29-Spiromics-47117390/Markups/88-Adam/S-Final-2-label.nrrd"
LabelPath = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-29-Spiromics-47117390/Markups/88-Adam/(NATE-DELETE)S-Final-2-label.nii.gz"

nifti_data = nib.load()
avmlabel = nifti_data.get_fdata()

OP = .9
DecimationFactor = 2

# Initialize case_folder before attempting to extract it
case_folder_pattern = re.compile(r'Case-\d+')
match = case_folder_pattern.search(CasePath)

if match:
    case_folder = match.group()
else:
    raise ValueError("No valid 'Case-N' folder name found in the path.")

# Output the extracted case folder for debugging
print(f"Extracted case folder: {case_folder}")

figure_name = case_folder  # Use extracted case folder as the figure name

# Create a figure with a specific name and size
fig = mlab.figure(figure_name, bgcolor=(1, 1, 1))

obj = (avmlabel == 1).astype(int)
mlab.contour3d(obj[::DecimationFactor, ::DecimationFactor, ::DecimationFactor], contours=[0.5], color=(0, 151/255, 206/255), opacity=OP)
obj = (avmlabel == 2).astype(int)
mlab.contour3d(obj[::DecimationFactor, ::DecimationFactor, ::DecimationFactor], contours=[0.5], color=(216/255, 101/255,  79/255), opacity=OP)

camera = mlab.gcf().scene.camera
mlab.gcf().scene.camera.position = [107, 128, 613]
mlab.gcf().scene.camera.focal_point = [149, 128, 148]
mlab.gcf().scene.camera.view_up = [0, 1, 0]

snapshot_directory = "z:/Q- Figures&Papers/1-T32-Poster/Nate/CaseFileCorrection/"

# Combine the snapshot directory with the new case folder to create the directory path
case_directory_path = os.path.join(snapshot_directory, case_folder)

# Check if the directory exists, and if not, create it
if not os.path.exists(case_directory_path):
    os.makedirs(case_directory_path)

# Combine the case directory path with the figure name to create the snapshot path
snapshot_path = os.path.join(case_directory_path, f"{figure_name}.png")

# Save a snapshot of the Mayavi figure
mlab.savefig(snapshot_path)

# Close the Mayavi figure
mlab.close()

# Optionally, you can display a message indicating that the snapshot has been saved
print(f"Snapshot saved to {snapshot_path}")


In [17]:
# GOLD STANDARD TO DISPLAY MAYAVI VIEWERS WORKS WITH NII.GZ

from mayavi import mlab
import numpy as np
import nibabel as nib

LabelPath = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-29-Spiromics-47117390/Markups/88-Adam/(NATE-DELETE)S-Final-2-label.nii.gz"

nifti_data = nib.load(LabelPath)
avmlabel = nifti_data.get_fdata()

OP = .9
DecimationFactor = 2

figure_name = LabelPath.split('/')[-4]  # Assuming '/' is the path separator

# Create a figure with a specific name and size
fig = mlab.figure(figure_name, size=(2500, 1400), bgcolor=(1, 1, 1))

obj = (avmlabel == 1).astype(int)
mlab.contour3d(obj[::DecimationFactor,::DecimationFactor,::DecimationFactor], contours=[0.5], color=(0      , 151/255, 206/255), opacity=OP)
obj = (avmlabel == 2).astype(int)
mlab.contour3d(obj[::DecimationFactor,::DecimationFactor,::DecimationFactor], contours=[0.5], color=(216/255, 101/255,  79/255), opacity=OP)

camera = mlab.gcf().scene.camera
mlab.gcf().scene.camera.position = [88.01952661, 330.50405307, 1043.54742071]
mlab.gcf().scene.camera.focal_point = [200, 160.5, 159]
mlab.gcf().scene.camera.view_up = [0, 1, 0]

mlab.show()


In [6]:
# GOLD STANDARD TO DISPLAY MAYAVI VIEWERS WORKS WITH NRRD

from mayavi import mlab
import numpy as np
import nrrd

def update_camera_info(obj, evt):
    camera = mlab.gcf().scene.camera
    print("Camera Position:", camera.position)
    print("Camera Focal Point:", camera.focal_point)
    print("Camera View Up:", camera.view_up)

LabelPath = "z:/Q- Figures&Papers/1-T32-Poster/Nate/101-3cm^3-Segmentation.seg.nrrd"

avmlabel, metadata = nrrd.read(LabelPath)


OP = 1
DecimationFactor = 1

figure_name = LabelPath.split('/')[-4]  # Assuming '/' is the path separator

# Create a figure with a specific name and size
fig = mlab.figure(figure_name, bgcolor=(1, 1, 1))

# obj = (avmlabel == 1).astype(int)
# mlab.contour3d(obj[::DecimationFactor,::DecimationFactor,::DecimationFactor], contours=[0.5], color=(0      , 151/255, 206/255), opacity=OP)
obj = (avmlabel == 2).astype(int)
mlab.contour3d(obj[::DecimationFactor,::DecimationFactor,::DecimationFactor], contours=[0.5], color=(216/255, 101/255,  79/255), opacity=OP)

# camera = mlab.gcf().scene.camera
# mlab.gcf().scene.camera.position = [107, 128, 613]
# mlab.gcf().scene.camera.focal_point = [149, 128, 148]
# mlab.gcf().scene.camera.view_up = [0, 1, 0]
# mlab.gcf().scene.interactor.add_observer('MouseMoveEvent', update_camera_info)
mlab.show()


In [15]:
#100 GOLD STANDARD CODE ALL IN ONE

# Import necessary libraries
import os
import SimpleITK as sitk
import numpy as np
from docx import Document
import time
from datetime import datetime
import nrrd
from mayavi import mlab
from docx.shared import Inches, Pt, RGBColor
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
import re
from docx.oxml.ns import qn
from docx.enum.text import WD_COLOR_INDEX
from docx.oxml import OxmlElement

HIGHLIGHT_COLORS = {
    'yellow': WD_COLOR_INDEX.YELLOW,
    'green': WD_COLOR_INDEX.BRIGHT_GREEN,
    'cyan': WD_COLOR_INDEX.TURQUOISE,
    'magenta': WD_COLOR_INDEX.PINK,
    'blue': WD_COLOR_INDEX.BLUE,
    'red': WD_COLOR_INDEX.RED,
    'none': WD_COLOR_INDEX.AUTO  # No highlighting
}


def add_horizontal_line(doc):
    """Adds a horizontal line to the Word document."""
    # Create a new paragraph for the horizontal line
    para = doc.add_paragraph()
    para_format = para.paragraph_format
    para_format.space_after = Pt(0)  # Reduce space after the paragraph
    para_format.space_before = Pt(0)
    p = para._p  # Access the underlying XML element of the paragraph
    pPr = p.get_or_add_pPr()  # Access paragraph properties

    # Create a border element
    pBdr = OxmlElement('w:pBdr')

    # Create a bottom border element
    bottomBdr = OxmlElement('w:bottom')
    bottomBdr.set(qn('w:val'), 'single')
    bottomBdr.set(qn('w:sz'), '4')  # Size of the border
    bottomBdr.set(qn('w:space'), '1')  # Space between border and text
    bottomBdr.set(qn('w:color'), 'auto')  # Border color

    # Add the bottom border to the border element
    pBdr.append(bottomBdr)

    # Add the border element to the paragraph properties
    pPr.append(pBdr)
def set_section_columns(section, num_columns, space_cm, separator=True):
    """ Set up the number of columns in a Word document section.
    
    Args:
    section (docx.section.Section): The section to modify.
    num_columns (int): The number of columns to set.
    space_cm (float): The spacing between columns in centimeters.
    separator (bool): Whether to add a line between columns.
    """
    sectPr = section._sectPr
    cols = sectPr.xpath('./w:cols')[0]
    cols.set(qn('w:num'), str(num_columns))
    cols.set(qn('w:space'), str(int(space_cm * 20)))  # Word space units are 1/20th of a point
    if separator:
        cols.set(qn('w:sep'), '1')
    else:
        cols.set(qn('w:sep'), '0')

COLORS_DICT = {
    'blue': (79, 129, 189),
    'green': (155, 187, 89),
    'red': (250, 0, 0,),
    'orange': (237, 125, 49),
    'white': (255, 255, 255)
    # Add more colors as needed
}

# Initialize a Document object for Word file output
doc = Document()
# doc = Document("z:/Q- Figures&Papers/1-T32-Poster/Nate/CaseFileCorrection/CaseFileCorrection.docx")
num_columns = 3  # or 3 for three columns
space_between_columns_cm = .5  # Adjust the spacing as necessary
set_section_columns(doc.sections[0], num_columns, space_between_columns_cm)


#2 Function to write data to Word document
##############################################################################################################
def write_to_word(text, bold=False, indent_level=0, font_color=None, alignment=None, highlight=None, label_file=False):
    paragraph = doc.add_paragraph()
    paragraph_format = paragraph.paragraph_format
    paragraph_format.space_after = Pt(0)
    paragraph_format.space_before = Pt(0)
    paragraph_format.left_indent = Pt(12 * indent_level)
    if label_file and 'Voxel Count' in text:
        font_color = 'green'

    # Check if the text contains a colon and a highlight color is specified
    if ':' in text and highlight:
        # Split the text into label and values
        label, values = text.split(':', 1)
        label_run = paragraph.add_run(label + ':')
        label_run.font.name = 'Courier New'
        label_run.font.size = Pt(6)
        if font_color:
            # Apply the font color if specified
            r, g, b = COLORS_DICT.get(font_color, (0, 0, 0))
            label_run.font.color.rgb = RGBColor(r, g, b)
        
        # Now create a new run for the values which will be highlighted
        values_run = paragraph.add_run(values)
        values_run.font.name = 'Courier New'
        values_run.font.size = Pt(6)
        if bold:
            values_run.bold = True

        # Set the highlight color for values
        if highlight in HIGHLIGHT_COLORS:
            values_run.font.highlight_color = HIGHLIGHT_COLORS[highlight]
            if HIGHLIGHT_COLORS[highlight] == WD_COLOR_INDEX.BLUE:
                # If highlight is blue, set font color to white for values
                values_run.font.color.rgb = RGBColor(255, 255, 255)
    else:
        # If there's no colon or no highlight, just treat it as a normal run
        run = paragraph.add_run(text)
        run.font.name = 'Courier New'
        run.font.size = Pt(6)
        if bold:
            run.bold = True

        # Set the font color
        if font_color:
            if isinstance(font_color, tuple) and len(font_color) == 3:
                run.font.color.rgb = RGBColor(*font_color)
            else:
                r, g, b = COLORS_DICT.get(font_color, (0, 0, 0))
                run.font.color.rgb = RGBColor(r, g, b)

        # Set the highlight color for the run
        if highlight in HIGHLIGHT_COLORS:
            run.font.highlight_color = HIGHLIGHT_COLORS[highlight]
            if HIGHLIGHT_COLORS[highlight] == WD_COLOR_INDEX.BLUE:
                # If highlight is blue, set font color to white
                run.font.color.rgb = RGBColor(255, 255, 255)

    # Set paragraph alignment if specified
    if alignment:
        paragraph.alignment = alignment

#3
def add_image_to_word(image_path, width=None):
    try:
        # Create a new paragraph for the image
        image_paragraph = doc.add_paragraph()
        image_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

        # Add an image to the Word document
        if width:
            run = image_paragraph.add_run()
            run.add_picture(image_path, width=Inches(width))
        else:
            run = image_paragraph.add_run()
            run.add_picture(image_path)  # Automatically uses the image's size

    except Exception as e:
        print(f"An error occurred while adding the image: {e}")

#4 Function to Get Image Details
def print_image_details(image, data, full_item_path):
    # Extracting size and spacing information
    size = image.GetSize()
    spacing = image.GetSpacing()
    size_output = f"Dimensions: {size[0]} x {size[1]} x {size[2]}"
    spacing_output = f"Spacing: {spacing[0]:.2f} x {spacing[1]:.2f} x {spacing[2]:.2f} mm"
    
    # Output the size and spacing information
    print(size_output)
    print(spacing_output)
    write_to_word(size_output, indent_level=0, alignment=WD_PARAGRAPH_ALIGNMENT.CENTER, highlight='yellow')
    write_to_word(spacing_output, indent_level=0, alignment=WD_PARAGRAPH_ALIGNMENT.CENTER, highlight='yellow')

#5 Extract segment information for labels
def extract_segments_info_for_labels(file_path):
    try:
        image = sitk.ReadImage(file_path)
        data = sitk.GetArrayFromImage(image)
        unique_values = np.unique(data)
        segment_names = [f"Segment_{value}" for value in unique_values]
        label_values = list(unique_values)
        num_segments = len(label_values)
        voxel_counts = {value: np.sum(data == value) for value in unique_values}

        return num_segments, segment_names, label_values, voxel_counts
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        write_to_word(f"An error occurred: {str(e)}")
        return None, None, None
    
#6 Map Color to Label Value
def map_color_to_label(metadata, color):
    # Convert color tuple from 0-1 to 0-255 range for comparison
    color_255 = tuple(int(c * 255) for c in color)
    # Search in metadata for the label with the given color
    for key, value in metadata.items():
        if key.endswith("_Color"):
            # Extract the RGB values and convert to int in range 0-255
            meta_color = tuple(int(float(v) * 255) for v in value.split())
            if meta_color == color_255:
                # Find the label value associated with this color
                label_key = key.replace('_Color', '_LabelValue')
                return int(metadata[label_key])
    return None

#7 Check segment names
def check_segment_names(metadata, case_path):
    segment_names = []
    for key in sorted(metadata.keys()):
        if key.endswith("_Name"):
            segment_names.append(metadata[key])

    # if len(segment_names) > 3:
    #     print(f"Warning: More than three segments found in {case_path}. Skipping file.")
    #     return False

    if len(segment_names) >= 2:
        if not segment_names[0].startswith('A') or not segment_names[1].startswith('V'):
            print(f"Warning: The segments may be named wrong for {case_path}")
    else:
        print(f"Warning: Not enough segments found in {case_path}")

    return True

#8 Visualization
def visualize_nrrd_and_save_snapshot(case_path, snapshot_directory):
    # try:
        avmlabel, metadata = nrrd.read(case_path)

        OP = 1
        DecimationFactor = 2
        use_label_value = case_path.endswith('label.nrrd') or case_path.endswith('Label.nrrd')

        if use_label_value:
            # When using label values, set the segments based on the label value
            Unraveled_A = (avmlabel == 1).astype(int)
            Unraveled_V = (avmlabel == 2).astype(int)
            color_a = (0, 0.592157, 0.807843)  # Assuming these are your default colors
            color_v = (0.847059, 0.396078, 0.309804)
        else:
            if not check_segment_names(metadata, case_path):
                return
            # Define the colors for 'A' and 'V' segments
            color_a = (0, 0.592157, 0.807843)  # Assuming these are your default colors
            color_v = (0.847059, 0.396078, 0.309804)
        # Find the label values that correspond to the colors
            label_a = map_color_to_label(metadata, color_a)
            label_v = map_color_to_label(metadata, color_v)

            Unraveled_A = (avmlabel == label_a).astype(int)
            Unraveled_V = (avmlabel == label_v).astype(int)

        # Create Unraveled seagments for 'A' and 'V'
        
        # Now use 'Unraveled_A' and 'Unraveled_V' to visualize in Mayavi
        figure_name = os.path.basename(case_path).split('.')[0]
        fig = mlab.figure(figure_name, bgcolor=(1, 1, 1))

        if Unraveled_A.any():
            mlab.contour3d(Unraveled_A[::DecimationFactor, ::DecimationFactor, ::DecimationFactor],
                    contours=[0.5], color=color_a, opacity=OP)

        # Visualize the Unraveled_V segment
        if Unraveled_V.any():
            mlab.contour3d(Unraveled_V[::DecimationFactor, ::DecimationFactor, ::DecimationFactor],
                    contours=[0.5], color=color_v, opacity=OP)

        # Camera and snapshot settings
        camera = mlab.gcf().scene.camera
        camera.position = [107, 140, 613]
        camera.focal_point = [175, 128, 148]
        camera.view_up = [0, 1, 0]
        camera.clipping_range = (1, 1000)

        path_parts = case_path.split(os.sep)  # Split the path into parts
        case_index = [i for i, part in enumerate(path_parts) if part.startswith('Case-')][0]  # Find the index of the part that starts with 'Case-'
        case_name = path_parts[case_index]  # Extract the case name

        # Constructing the directory and file names
        case_directory_name = case_name  # Use the case name as the directory name
        final_file_name = f"{case_name}-{path_parts[-1].replace('.nrrd', '.png')}"  # Construct the file name

        # Directory creation and snapshot saving
        case_directory_path = os.path.join(snapshot_directory, case_directory_name)
        if not os.path.exists(case_directory_path):
            os.makedirs(case_directory_path)

        snapshot_path = os.path.join(case_directory_path, final_file_name) 
        mlab.savefig(snapshot_path)
        
        mlab.close()
        print(f"Snapshot saved to {snapshot_path}")
        return case_path, snapshot_path 

#9 Print Filtered Nrrd Header
def print_filtered_nrrd_header(image, data=None):
    try:
        keys = image.GetMetaDataKeys()
        segment_count = 0
        segment_data = {}
        
        for key in keys:
            if "Segment" in key:
                segment_info = key.split('Segment')[-1]
                if segment_info[0].isdigit():
                    seg_num, attribute = segment_info.split('_', 1)
                    if seg_num not in segment_data:
                        segment_data[seg_num] = {}
                        segment_count += 1
                    if attribute in ['Color', 'ID', 'LabelValue', 'Name']:
                        value = image.GetMetaData(key)
                        segment_data[seg_num][attribute] = value
        
        output_text = f"Number of Segments: {segment_count}"
        print(output_text)
        write_to_word(output_text, indent_level=0, alignment=WD_PARAGRAPH_ALIGNMENT.CENTER)

        if data is not None:
            total_voxel_count = f"{np.sum(data >= 0):,}"  # Sum of all voxels in the image
            output_text = f"Total Voxel Count: {total_voxel_count}"
            print(output_text)
            # write_to_word(output_text, indent_level=8, font_color='orange')

        for seg_num, attributes in segment_data.items():
            segment_name_text = f"Segment {seg_num} Name: {attributes.get('Name', 'N/A')} ■"  # Concatenate the '■' character here
            # segment_id = f"Segment {seg_num} ID: {attributes.get('ID', 'N/A')}"
            # segment_color = f"Segment {seg_num} Color: {attributes.get('Color', 'N/A')}"
            segment_label_value = f"LabelValue: {attributes.get('LabelValue', 'N/A')}"

            if attributes.get('Color', 'N/A') != 'N/A':
                color_value = attributes.get('Color', 'N/A')
                r, g, b = [int(float(x) * 255) for x in color_value.split()]

            # Split the segment name write into two: one for the name and one for the '■' with color.
            write_to_word(segment_name_text, indent_level=0, font_color=(r,g,b), alignment=WD_PARAGRAPH_ALIGNMENT.CENTER)  # Write the segment name without '■'
            
            # Add other attributes
            # write_to_word(segment_id, indent_level=10)
            # write_to_word(segment_color, indent_level=0)
            write_to_word(segment_label_value, indent_level=0, alignment=WD_PARAGRAPH_ALIGNMENT.CENTER, highlight='yellow')
            label_value = attributes.get('LabelValue')

            if label_value:
                label_value = int(label_value)  # Ensure it's an integer
                voxel_count = f"{np.sum(data == label_value):,}"  # Count and format with commas
                voxel_count_text = f"Voxel Count: {voxel_count}"
                write_to_word(voxel_count_text, indent_level=0, alignment=WD_PARAGRAPH_ALIGNMENT.CENTER, highlight='yellow')
            write_to_word(" ")

    except Exception as e:
        output_text = f"An error occurred: {str(e)}"
        print(output_text)
        write_to_word(output_text)

#10 Process Nrrd File in folder 
def process_nrrd_files_in_folder(folder_path): 
    add_horizontal_line(doc)
    # Extract case ID from the folder path
    

# Initialize the dictionary to keep track of added images
    pattern = r"Case-\d+"
    match = re.search(pattern, folder_path)
    case_id = match.group()
    files_and_subdirs = os.listdir(folder_path)
    output_text = f"File: {case_id}"
    print(output_text)
    write_to_word(output_text, bold=True, alignment=WD_PARAGRAPH_ALIGNMENT.CENTER, highlight='red')
    # write_to_word(" ")

    not_processed = [item for item in files_and_subdirs if not (item.startswith("S-") and item.endswith('.nrrd'))]
    if not_processed:
        output_text = f"Files NOT being processed: {len(not_processed)}"
        print(output_text)
        write_to_word(output_text, bold=True, indent_level=0, font_color='blue', alignment=WD_PARAGRAPH_ALIGNMENT.CENTER)
        for index, item in enumerate(not_processed, 1): # Start the index from 1
            output_text = f"{index}. {item}" # Replace dashes with numbers
            print(output_text)
            write_to_word(output_text, indent_level=0, font_color='blue', alignment=WD_PARAGRAPH_ALIGNMENT.CENTER)
        write_to_word(" ")
    else:
        output_text = "All files in this folder will be processed."
        print(output_text)
        write_to_word(output_text, bold=True, indent_level=0, font_color='blue', alignment=WD_PARAGRAPH_ALIGNMENT.CENTER)

    to_process = [item for item in files_and_subdirs if item.startswith("S-") and item.endswith('.nrrd')]
    if to_process:
        output_text = f"Files being processed: {len(to_process)}" # Number of items being processed

        print(output_text)
        write_to_word(output_text, bold=True, indent_level=0, font_color='blue', alignment=WD_PARAGRAPH_ALIGNMENT.CENTER)
        
        for index, item in enumerate(to_process, 1): # Start the index from 1
            output_text = f"{index}. {item}" 
            print(output_text)
            # write_to_word(output_text, indent_level=0, font_color='blue')

    for item in files_and_subdirs:
        if item.startswith("S-"):
            full_item_path = os.path.join(folder_path, item)
            if os.path.isfile(full_item_path) and item.endswith('.nrrd'):
                creation_time = os.path.getctime(full_item_path)
                formatted_creation_time = datetime.fromtimestamp(creation_time).strftime('%Y/%m/%d %H:%M')
                image = sitk.ReadImage(full_item_path)
                data = sitk.GetArrayFromImage(image)

                # Determine the font color
                if "label" in item.lower():
                    font_color_val = 'green'
                else:
                    font_color_val = None  # default color
                output_text = f"{item}"
                output_filename_NIFTI = item.replace("S-", "SS-").replace(".nrrd", ".nii.gz").replace(".seg.",".")
                output_filename_save = "→ ", output_filename_NIFTI
                print(output_text)
                write_to_word(output_text, bold=True, indent_level=0, font_color='white', alignment=WD_PARAGRAPH_ALIGNMENT.CENTER, highlight='blue')
                write_to_word(output_filename_save, bold=True, indent_level=0, font_color='red', alignment=WD_PARAGRAPH_ALIGNMENT.CENTER)
                write_to_word(f"Created on: {formatted_creation_time}", indent_level=0, alignment=WD_PARAGRAPH_ALIGNMENT.CENTER)
                image = sitk.ReadImage(full_item_path)
                data = sitk.GetArrayFromImage(image)
                print_image_details(image, data, full_item_path)
                                         






                new_case_path, snapshot_directory_of_png = visualize_nrrd_and_save_snapshot(full_item_path, snapshot_directory)
                add_image_to_word(snapshot_directory_of_png, width=1)     
                
                if "label" in item.lower():
                    num_segments, segment_names, label_values, voxel_counts = extract_segments_info_for_labels(full_item_path)
                    if num_segments is not None and num_segments > 0:
                        output_text = f"Number of Segments: {num_segments}"
                        print(output_text)
                        write_to_word(output_text, indent_level=0, font_color=font_color_val, alignment=WD_PARAGRAPH_ALIGNMENT.CENTER)
                       
                        for i in range(num_segments):
                            
                            segment_output = [
                                f"Segment {i + 1}:",
                                f"Label Value: {label_values[i]} \n"
                                f"Voxel Count: {format(voxel_counts[label_values[i]], ',')}"
                            ]
                            for line in segment_output:
                                print(line)
                                write_to_word(line, indent_level=0, font_color=font_color_val, alignment=WD_PARAGRAPH_ALIGNMENT.CENTER, highlight='yellow', label_file=True)
                        
                        # write_to_word(" ")
                else:
                    print_filtered_nrrd_header(image, data)
        add_horizontal_line(doc)


def search_and_process(base_path, case_numbers, max_cases=5):
    processed_cases = 0
    # Convert case_numbers to a set for faster lookup
    case_numbers_set = set(case_numbers)

    for root, dirs, files in os.walk(base_path):
        # dirs.sort()  # Optional: Sort the directories if you want to process them in a specific order
        for dir_name in dirs:
            if processed_cases >= max_cases:
                break
            if dir_name == "1-Duplicates" or not dir_name.startswith("Case-"):
                continue

            case_num_str = dir_name.split('-')[1]  # Extract the numerical part as a string
            if case_num_str.isdigit():  # Ensure the extracted part is a digit
                case_num = int(case_num_str)
                if case_num in case_numbers_set:
                    # Your processing logic here
                    for sub_root, sub_dirs, sub_files in os.walk(os.path.join(root, dir_name)):
                        for sub_dir_name in sub_dirs:
                            if sub_dir_name.startswith("88-"):
                                process_nrrd_files_in_folder(os.path.join(sub_root, sub_dir_name))
                                doc.save('z:/Q- Figures&Papers/1-T32-Poster/Nate/CaseFileCorrection/CaseFileCorrection(V18).docx')
                                processed_cases += 1
                                if processed_cases >= max_cases:
                                    return





#12 Define the directory path and call all functions previously
base_folder_path = "z:/D-Images/"
snapshot_directory = "z:/Q- Figures&Papers/1-T32-Poster/Nate/CaseFileCorrection/"
vx3 = "z:/Q- Figures&Papers/1-T32-Poster/Nate/ImageCreationFile/QC-Images/"
image_files = sorted(os.listdir(vx3))
image_paths = [os.path.join(vx3, file) for file in image_files if file.endswith('.png')]
case_numbers = [30, 32, 34, 36, 38]
#case_numbers = [39, 42, 43, 44, 46, 47, 48, 49, 50, 53, 54, 55, 56, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 76, 77, 78, 79, 81, 82, 83, 84, 86]

search_and_process(base_folder_path, case_numbers)
# Save the document
doc.save('z:/Q- Figures&Papers/1-T32-Poster/Nate/CaseFileCorrection/CaseFileCorrection(V18).docx')

File: Case-30
Files NOT being processed: 6
1. F-1-R.mrk.json
2. F-2-Problems.mrk.json
3. F-3-Soheils.mrk.json
4. Trash
5. Vx3-2.nrrd
6. Vx3.nrrd
Files being processed: 11
1. S-Final-1.nrrd
2. S-Final-2.nrrd
3. S-Final-3-label.nrrd
4. S-Final-3.nrrd
5. S-Segmentation-1.nrrd
6. S-Segmentation-2.nrrd
7. S-Segmentation-3.nrrd
8. S-Segmentation-4.nrrd
9. S-Segmentation-5.nrrd
10. S-Segmentation-6.nrrd
11. S-Segmentation-7.nrrd
S-Final-1.nrrd
Dimensions: 831 x 603 x 733
Spacing: 0.38 x 0.38 x 0.38 mm
Snapshot saved to z:/Q- Figures&Papers/1-T32-Poster/Nate/CaseFileCorrection/Case-30-Spiromics-46974609\Case-30-Spiromics-46974609-S-Final-1.png
Number of Segments: 3
Total Voxel Count: 367,301,169
S-Final-2.nrrd
Dimensions: 831 x 603 x 733
Spacing: 0.38 x 0.38 x 0.38 mm
Snapshot saved to z:/Q- Figures&Papers/1-T32-Poster/Nate/CaseFileCorrection/Case-30-Spiromics-46974609\Case-30-Spiromics-46974609-S-Final-2.png
Number of Segments: 3
Total Voxel Count: 367,301,169
S-Final-3-label.nrrd
Dimensions:

In [17]:
import re
from docx import Document

def find_cases_in_docx(docx_path):
    doc = Document(docx_path)
    case_pattern = re.compile(r'Case-\d{2}')

    cases_found = {}  # Dictionary to hold 'case: [positions]'
    
    for i, paragraph in enumerate(doc.paragraphs):
        for match in case_pattern.finditer(paragraph.text):
            case = match.group()
            if case in cases_found:
                cases_found[case].append(i)  # Assuming 'i' as a proxy for position, not page number
            else:
                cases_found[case] = [i]
    
    return cases_found

# Example usage
docx_path = 'z:/Q- Figures&Papers/1-T32-Poster/Nate/CaseFileCorrection/CaseFileCorrectionFINALREPORT.docx'
cases_found = find_cases_in_docx(docx_path)
for case, positions in cases_found.items():
    print(f"{case} found at positions: {positions}")


Case-01 found at positions: [1]
Case-02 found at positions: [214]
Case-03 found at positions: [337]
Case-04 found at positions: [444]
Case-06 found at positions: [553]
Case-07 found at positions: [666]
Case-09 found at positions: [865]
Case-10 found at positions: [986]
Case-11 found at positions: [1109]
Case-12 found at positions: [1242]
Case-14 found at positions: [1359]
Case-15 found at positions: [1586]
Case-16 found at positions: [1667]
Case-17 found at positions: [1790]
Case-18 found at positions: [1859]
Case-19 found at positions: [1930]
Case-20 found at positions: [1995]
Case-21 found at positions: [2146]
Case-23 found at positions: [2445]
Case-27 found at positions: [2726]
Case-37 found at positions: [2915]
Case-87 found at positions: [3258]
Case-88 found at positions: [3445]
Case-89 found at positions: [3710]
Case-90 found at positions: [3821]
Case-91 found at positions: [3978]
Case-93 found at positions: [4125]
Case-94 found at positions: [4190]
Case-95 found at positions: [4